<a href="https://colab.research.google.com/github/hmghaly/word_align/blob/main/walign2022_NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from google.colab import drive
import os
drive.mount('/content/drive')
cwd='/content/drive/MyDrive/tr_ar' #directory where we keep the data
if not os.path.exists(cwd): os.makedirs(cwd)  #if the directory doesn't exist, create it
os.chdir(cwd) #change current work directory to the directory where we keep the data
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/tr_ar


In [2]:
#Clone the github codebase
!rm -r code_utils #first delete the code_utils folder, then clone from the codebase (very crude till we figure out using github from colab)
!git clone https://github.com/hmghaly/word_align.git code_utils

Cloning into 'code_utils'...
remote: Enumerating objects: 905, done.
remote: Counting objects: 100% (371/371), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 905 (delta 211), reused 366 (delta 208), pack-reused 534
Receiving objects: 100% (905/905), 26.43 MiB | 9.54 MiB/s, done.
Resolving deltas: 100% (235/235), done.


In [3]:
from code_utils.general import *
from code_utils.align_utils import *
from code_utils.arabic_lib import *

from itertools import groupby
from collections import Counter
ar_counter_dict=load_counts("ar_count.txt", tmp_count_dict={})

def cur_ar_tok_fn(text):
  cur_toks=tok(text)
  return tok_ar(cur_toks,ar_counter_dict)
print("loaded necessary functions")

loaded necessary functions


#Loading and indexing sentences
new approach - just indexing and word alignment for all the data

## Loading functions

In [60]:
#main indexing functions
import os, time, json

def file_len(fpath0): #get the number of lines in the cache file
  if not os.path.exists(fpath0): return 0
  with open(fpath0) as fopen:
    for i0,f0 in enumerate(fopen): pass
  return i0+1


class sent_cls:
  def __init__(self,raw_sent0="",raw_tokens0=[],tokens0=[],map0=[]) -> None:
      self.raw_sent=raw_sent0
      self.raw_tokens=raw_tokens0
      self.tokens=tokens0 #filtered tokens
      self.map=map0 #mapping of filtered tokens to raw tokens #raw_tokens,map

class indexing: #get a list of sentences, outputs indexes
  def __init__(self,raw_sentences0,tok_function, params0={}):
    lang=params0.get("lang","en")
    stop_words=params0.get("stop_words",[]) #excluded words
    ignore_punc=params0.get("ignore_punc",False) #skip punctuation or not
    ignore_ar_pre_suf=params0.get("ignore_ar_pre_suf",False) #ignore arabic prefixes and suffixes
    remove_al=params0.get("remove_al", True) #remocve alif laam in Arabic
    index_words=params0.get("index_words", True) #keep the index of each word in a sentence, not just the sentence number
    lower=params0.get("lower", True) #make all words lower case or not
    stemming=params0.get("stemming", False) #stem each word or not
    self.fwd_index=[]
    self.all_tok_sentences=[]
    self.all_tok_original_sentences=[]
    self.all_mappings=[]
    self.sent_obj_list=[]
    self.inv_index={}
    for sent_i,sent0 in enumerate(raw_sentences0):
      #if sent_i%5000==0: print(sent_i)
      tokens=tok_function(sent0)
      #tokens,cur_mapping,original_tokens=filter_tokens(tokens, lang,stop_words, ignore_punc,ignore_ar_pre_suf,remove_al,index_words,lower,stemming)
      tokens,cur_mapping,original_tokens=filter_tokens(tokens, params0)
      self.all_tok_original_sentences.append(original_tokens)
      self.all_mappings.append(cur_mapping)
      enumerated_toks=[(v,sent_i,s_i) for s_i,v in enumerate(tokens)]
      cur_sent_obj0=sent_cls(sent0,original_tokens,tokens,cur_mapping)
      self.sent_obj_list.append(cur_sent_obj0)
      #cur_sent_obj0
      self.all_tok_sentences.append(tokens)
      self.fwd_index.extend(enumerated_toks)
    self.fwd_index.sort()
    grouped=[(key,[v[1:] for v in list(group)]) for key,group in groupby(self.fwd_index,lambda x:x[0])]
    self.inv_index=dict(iter(grouped))

def filter_tokens(token_list0, params0={}):
#def filter_tokens(token_list0, lang="en",stop_words=[], ignore_punc=False,ignore_ar_pre_suf=False,remove_al=True,index_words=True,lower=True,stemming=False):
  tokens_original=list(token_list0)
  tokens_copy_enum=[(i,v) for i,v in enumerate(token_list0)]   
  ignore_punc=params0.get("ignore_punc",False)
  ignore_ar_pre_suf=params0.get("ignore_ar_pre_suf",False)
  remove_al=params0.get("remove_al",True)
  lower=params0.get("lower",True)
  stemming=params0.get("stemming",False)
  lang=params0.get("lang","en")
  stop_words=params0.get("stop_words",[])
  #if lang=="ar": tokens=tok_ar(tokens,count_dict) #tok_ar(tokens)
  if ignore_punc: tokens_copy_enum=[(i,v) for i,v in tokens_copy_enum if not is_punct(v)]
  if ignore_ar_pre_suf: tokens_copy_enum=[(i,v) for i,v in tokens_copy_enum if not v.startswith("ـ") and not v.endswith("ـ")]
  if stop_words: tokens_copy_enum=[(i,v) for i,v in tokens_copy_enum if not v.lower() in stop_words]
  if remove_al: tokens_copy_enum=[(i,v.replace("ال_","")) for i,v in tokens_copy_enum if not v.lower() in stop_words]
  tokens_out=[v[1] for v in tokens_copy_enum]
  if lower: tokens_out=[v.lower() for v in tokens_out]
  cur_mapping=[v[0] for v in tokens_copy_enum]
  return tokens_out,cur_mapping, tokens_original

def get_tok_loc(tok_list):
  tok_locs_list=[]
  for i_,cur_tok in enumerate(tok_list):
    tok_locs_list.append(((cur_tok,),(i_,i_))) #to get unigrams and their locations
    if i_<len(tok_list)-1: tok_locs_list.append(((cur_tok,tok_list[i_+1]), (i_,i_+1))) #to get bigrams and their locations
  tok_locs_list.sort(key=lambda x:x[0])
  grouped=[(key,[v[1] for v in list(group)]) for key,group in groupby(tok_locs_list,lambda x:x[0])]
  return grouped

def get_rec_el_children(el0,el_child_dict0,el_list0=[]):
  cur_children0=el_child_dict0.get(el0,[])
  if cur_children0==[]: el_list0.append(el0)
  for ch0 in cur_children0:
    el_list0=get_rec_el_children(ch0,el_child_dict0,el_list0)
  return el_list0

def combine_els(el1,el2):
  src_span1,trg_span1=el1
  src_span2,trg_span2=el2
  x1_0,x1_1=src_span1
  x2_0,x2_1=src_span2
  y1_0,y1_1=trg_span1
  y2_0,y2_1=trg_span2
  min_x=min(x1_0,x1_1,x2_0,x2_1)
  max_x=max(x1_0,x1_1,x2_0,x2_1)
  min_y=min(y1_0,y1_1,y2_0,y2_1)
  max_y=max(y1_0,y1_1,y2_0,y2_1)
  new_src_span=(min_x,max_x)
  new_trg_span=(min_y,max_y)
  return (new_src_span,new_trg_span)

# def check_span_overlap(span1,span2):
#   span1_x1,span1_x2=span1
#   span2_x1,span2_x2=span2
#   d1=abs(span1_x2-span1_x1)
#   d2=abs(span2_x2-span2_x1)
#   max_x=max(span1_x1,span1_x2,span2_x1,span2_x2)
#   min_x=min(span1_x1,span1_x2,span2_x1,span2_x2)
#   d_total=max_x-min_x
#   overlapping=False
#   if d_total<=d1+d2: overlapping=True
#   return overlapping

def get_span_dist(span1,span2):
  span1_x1,span1_x2=span1
  span2_x1,span2_x2=span2
  d1=abs(span1_x2-span1_x1)
  d2=abs(span2_x2-span2_x1)
  max_x=max(span1_x1,span1_x2,span2_x1,span2_x2)
  min_x=min(span1_x1,span1_x2,span2_x1,span2_x2)
  d_total=max_x-min_x
  return d_total-d1-d2



# def overlaps(el1,el2,max_dist=10):
#   src_span1,trg_span1=el1
#   src_span2,trg_span2=el2
#   x1_0,x1_1=src_span1
#   x2_0,x2_1=src_span2
#   y1_0,y1_1=trg_span1
#   y2_0,y2_1=trg_span2
#   #if x2_0<=x1_1: return True
#   output=False
#   if check_span_overlap(src_span1,src_span2): output=True
#   if check_span_overlap(trg_span1,trg_span2): output=True
#   if get_span_dist(trg_span1,trg_span2)>max_dist: output=True

#   # min_x=min(x1_0,x1_1,x2_0,x2_1)
#   # max_x=max(x1_0,x1_1,x2_0,x2_1)
#   # min_y=min(y1_0,y1_1,y2_0,y2_1)
#   # max_y=max(y1_0,y1_1,y2_0,y2_1)
#   # new_src_span=(min_x,max_x)
#   # new_trg_span=(min_y,max_y)
#   return output

# def overlaps_OLD(span1,span2):
#   span1_i0,span1_i1=span1
#   span2_i0,span2_i1=span2
#   if span1_i0>=span2_i0 and span1_i0<=span2_i1: return True
#   if span1_i1>=span2_i0 and span1_i1<=span2_i1: return True
#   if span2_i0>=span1_i0 and span2_i0<=span1_i1: return True
#   if span2_i1>=span1_i0 and span2_i1<=span1_i1: return True
#   return False


def walign(src_obj0,trg_obj0,src_inv_dict0,trg_inv_dict0,walign_params0={}):
  min_retrieved_index_count=walign_params0.get("min_retrieved_index_count",3) #3
  max_dist=walign_params0.get("max_dist",10)
  min_ratio=walign_params0.get("min_ratio",0.05)
  max_sent_size=walign_params0.get("max_sent_size")
  n_epochs=walign_params0.get("n_epochs",20)
  output={}

  span_el_dict={} #which elements corresponds to this span - sorted by weight
  start_span_dict={} #show all the spans corresponding to a src location
  el_child_dict={} #show which elements are children of which
  el_wt_dict={} #show the weight of each element

  
  span_wt_dict={} #wight of each span -> the weight of the highest element it includes
  span_child_dict={} #which subspans this span consists of
  span_top_elms_dict={} #show the top elements for each span
  
  
  
  all_phrase_pairs=[]
  src0,trg0=src_obj0.tokens, trg_obj0.tokens
  original_src0,original_trg0=src_obj0.raw_tokens, trg_obj0.raw_tokens
  map_src0,map_trg0=src_obj0.map, trg_obj0.map
  #trg0=trg_obj0.tokens #raw_tokens,map
  #if len(src0)>35: return


  src0=['<s>']+src0+['</s>'] #first we pad the sentence tokens with start and end sentence markers
  trg0=['<s>']+trg0+['</s>'] #for both src and trg
  original_src0=['<s>']+original_src0+['</s>'] #first we pad the sentence tokens with start and end sentence markers
  original_trg0=['<s>']+original_trg0+['</s>'] #for both src and trg

  output["src"]=original_src0
  output["trg"]=original_trg0
  

  
  if max_sent_size!=None and len(src0)>max_sent_size: return output


  # self.src0=src0
  # self.trg0=trg0
  #print(src0)
  src_grouped=get_tok_loc(src0) #group all the locations for a list of consecutive tokens (src then trg)
  trg_grouped=get_tok_loc(trg0)
  src_side,trg_side=[],[] #mapping unigrams and bigrams to both their locations within the sentence and retrieved indexes from inverted dict
  src_locs_count_dict={} #how many times did the current token/bigram/phrase appeared in the src sent tokens
  trg_locs_count_dict={} #and same for trg
  for s_toks0,s_locs_grp0 in src_grouped: #retrive the indexes for each token sequence, map them to the locations
    src_indexes0= rtrv_idx(s_toks0,src_inv_dict0)
    s_toks_str0=" ".join(s_toks0) 
    src_side.append((s_toks_str0,s_locs_grp0,src_indexes0))
    src_locs_count_dict[s_toks_str0]=len(s_locs_grp0)
  for t_toks0,t_locs_grp0 in trg_grouped:
    trg_indexes0= rtrv_idx(t_toks0,trg_inv_dict0)
    t_toks_str0=" ".join(t_toks0)
    trg_side.append((t_toks_str0,t_locs_grp0,trg_indexes0))
    trg_locs_count_dict[t_toks_str0]=len(t_locs_grp0)

  #now starting the match the indexes of both src and trg token locations and get the corresponding rations/weights
  matching_list=[]
  all_elements=[] #to create elements (pairs of src-trg spans)
  tmp_ratio_dict={}
  for src_item0 in src_side: #examining all src tokens/bigrams with their locs/spans and retrieved indexes
    s_toks_str0,s_locs_grp0,src_indexes0=src_item0
    for trg_item0 in trg_side: #and get also all trg tokens/bigrams with locs/spand and retrieved indexes
      t_toks_str0,t_locs_grp0,trg_indexes0=trg_item0
      cur_ratio=0
      if s_toks_str0==t_toks_str0: cur_ratio=1.0 #if the src equals the target (e.g. numbers), the matching ratio is 1
      else: 
        if len(src_indexes0)<min_retrieved_index_count or len(trg_indexes0)<min_retrieved_index_count: continue #if the number of retrieved indexes is less than certain threshold, we keep the ratio as zero
        cur_ratio=idx_match(src_indexes0,trg_indexes0,5)
      if cur_ratio==0: continue
      matching_list.append(([s_toks_str0,s_locs_grp0],[t_toks_str0,t_locs_grp0], cur_ratio))

  #####
  #Now filtering the matching list
  matching_list.sort(key=lambda x:-x[-1])
  valid_list=[] #the final list of acceptable phrase pairs, their locs, and matching ratio
  for ml in matching_list:
    a,b,r=ml
    src_phrase0,src_locs=a #for the current src phrase and how many locations it occurs in the current sentence
    trg_phrase0,trg_locs=b #and same for trg phrase
    min_count=min(len(src_locs), len(trg_locs)) #and we get the minimum occurance between the src and trg phrase
    found_src_count=src_locs_count_dict[src_phrase0] #and then we check whether the src phrase was used before
    found_trg_count=trg_locs_count_dict[trg_phrase0] #and same for trg phrase
    if found_src_count<1 or found_trg_count<1: continue #if either was used before, their count would be zero or less, so we exclude current pair
    src_locs_count_dict[src_phrase0]=found_src_count-min_count #if not, we proceed, abd subtract the minimum count from the "unused" count of the current src phrase
    trg_locs_count_dict[trg_phrase0]=found_trg_count-min_count #and same for trg
    valid_list.append(ml)
  #Now we star processing the valid pairs, with their actual geometric spans and weights
  #first populate the main dictionaries: span_el_dict,  start_span_dict, el_child_dict, el_wt_dict
  span_el_list=[] #list of spans and elements
  all_src_spans=[]
  for vl in valid_list:
    #print(">>>", vl)
    a,b,cur_ratio=vl
    src_phrase0,src_locs=a #for the current src phrase and how many locations it occurs in the current sentence
    trg_phrase0,trg_locs=b #and same for trg phrase
    for sl0 in src_locs:
      if not sl0 in all_src_spans: all_src_spans.append(sl0)
      for tl0 in trg_locs:
        cur_el=(sl0,tl0) #an element is a tuple containing the span of src indexes and span of trg indexes
        el_wt_dict[cur_el]=cur_ratio #initially populating the element weight dict
        span_el_list.append((sl0,cur_el,cur_ratio))
  span_el_list.sort(key=lambda x:x[0])
  span_el_list_grouped=[(key,[v[1] for v in list(group)]) for key,group in groupby(span_el_list,lambda x:x[0])]
  all_src_spans.sort()
  all_src_spans_grouped=[(key,list(group)) for key,group in groupby(all_src_spans,lambda x:x[0])]
  for sp0,elements0 in span_el_list_grouped: span_el_dict[sp0]=elements0 #initially populating the span-elements dict
  for start_i0,spans0 in all_src_spans_grouped: start_span_dict[start_i0]=spans0 #initially poplating the start/spans dict
  top_score=0
  full_src_span0=(0,len(src0)-1)
  full_trg_span0=(0,len(trg0)-1)
  full_el=(full_src_span0,full_trg_span0)
  #all_top_scores=[]

  for ep0 in range(n_epochs):
    #continue
    cur_start_locs=sorted(list(set(start_span_dict.keys())))
    #print(ep0, "cur_start_locs",cur_start_locs)
    for i_,loc_i in enumerate(cur_start_locs):
      next_locs=cur_start_locs[i_+1:]
      cur_spans=start_span_dict[loc_i]
      next_spans=[]
      for nl in next_locs: next_spans.extend(start_span_dict[nl])
      #print(ep0,"cur_spans",cur_spans,"next_spans",next_spans)

      for cs in cur_spans: 
        cur_els=span_el_dict.get(cs,[])
        #print("$$$$$ cur_els",cur_els)
        for ns in next_spans:
          #if ns[0]<=cs[1]: continue #print("exclude:",cs,ns)
          x_dist=get_span_dist(cs,ns)
          #print("????", cs,ns, "x_dist",x_dist)
          if x_dist<1 or x_dist>max_dist: continue
          #else: print("++++", cs,ns)
          next_els=span_el_dict.get(ns,[])
          #print("#### next_els",next_els)
          for cur_el0 in cur_els:
            cur_wt0=el_wt_dict[cur_el0]
            cur_x_span,cur_y_span=cur_el0
            for next_el0 in next_els:
              if cur_el0==next_el0: continue
              next_x_span,next_y_span=next_el0
              y_dist=get_span_dist(cur_y_span,next_y_span)
              if y_dist<1 or y_dist>max_dist: continue #print("Excluded XXX: cur_el0",cur_el0, "next_el0",next_el0,"y_dist",y_dist)
              next_wt0=el_wt_dict[next_el0] 
              combined_wt0=cur_wt0+next_wt0
              combined_el0=combine_els(cur_el0,next_el0) #get_min_max(cur_el0,next_el0)
              found_combined_wt=el_wt_dict.get(combined_el0,0)
              #print(">>>", cur_el0, next_el0,"combined_wt0",combined_wt0,"found_combined_wt",found_combined_wt)       
              if combined_wt0>found_combined_wt:
                el_wt_dict[combined_el0]=combined_wt0
                el_child_dict[combined_el0]=[cur_el0,next_el0]
                tmp_src_span0=combined_el0[0]
                #print("tmp_src_span0",tmp_src_span0)
                found_span_elements=span_el_dict.get(tmp_src_span0,[])
                if not combined_el0 in found_span_elements: found_span_elements.append(combined_el0)
                found_span_elements.sort(key=lambda x:-el_wt_dict[x])
                span_el_dict[tmp_src_span0]=found_span_elements
                #print("span_el_dict[tmp_src_span0]",span_el_dict[tmp_src_span0])
                found_start_spans=start_span_dict.get(tmp_src_span0[0])
                if not tmp_src_span0 in found_start_spans: found_start_spans.append(tmp_src_span0)
                start_span_dict[tmp_src_span0[0]]=found_start_spans
                #print("start_span_dict[tmp_src_span0[0]]",start_span_dict[tmp_src_span0[0]])
                #print(ep0, "cur_el0",cur_el0, "next_el0",next_el0,"combined_el0",combined_el0, "combined_wt0",round(combined_wt0,4), "y_dist",y_dist)
        

    cur_score=el_wt_dict.get(full_el,0)
    wt_list=list(el_wt_dict.items())
    wt_list.sort(key=lambda x:-x[1])
    #print(ep0, wt_list[0], "cur_score",cur_score, "top",top_score)
    # for wl in wt_list[:5]:
    #   print(ep0, wl)
    # print("----")
    # print(all_top_scores)
    # all_top_scores.append(cur_score)

    if cur_score>top_score: top_score=cur_score
    elif cur_score==top_score and cur_score>0: break

  if el_wt_dict.get(full_el,0)>0:
    align_list=get_rec_el_children(full_el,el_child_dict,el_list0=[])
  else:
    top_el=wt_list[0][0]
    print("couldn't find full el:",full_el, "used the top el instead", top_el)
    align_list=get_rec_el_children(top_el,el_child_dict,el_list0=[])

  
  if align_list!=[]:
    # src0=src_obj0.tokens
    # trg0=trg_obj0.tokens
    output_align_list=[]

    # print("aligned")
    # print(src0)
    # print(trg0)
    # print(map_src0)
    # print(map_trg0)
    adj_src_map=[0]+[v+1 for v in map_src0]+[len(original_src0)-1]
    adj_trg_map=[0]+[v+1 for v in map_trg0]+[len(original_trg0)-1]
    # print(original_src0)
    # print(original_trg0)
    # print("adj_src_map",adj_src_map)
    # print("adj_trg_map",adj_trg_map)
    for a in align_list:
      src_span0,trg_span0=a
      x0,x1=src_span0
      y0,y1=trg_span0
      new_x0,new_x1=adj_src_map[x0],adj_src_map[x1]
      new_y0,new_y1=adj_trg_map[y0],adj_trg_map[y1]
      new_src_pan=new_x0,new_x1
      new_trg_span=new_y0,new_y1
      
      src_phrase0=src0[src_span0[0]:src_span0[1]+1]
      trg_phrase0=trg0[trg_span0[0]:trg_span0[1]+1]

      output_align_list.append((new_src_pan,new_trg_span))

      src_phrase1=original_src0[new_x0:new_x1+1]
      trg_phrase1=original_trg0[new_y0:new_y1+1]
    #   print(a,src_phrase0, trg_phrase0)
    #   print("src_phrase1",src_phrase1)
    #   print("trg_phrase1",trg_phrase1)
    #   print("---")
    # print("==============")
  #out["align"]=output_align_list
  output["align"]=output_align_list
  return output


##Starting Indexing/Alignment

In [ ]:
#Starting the experiment
exp_dir="exp00"
batch_size=100000
#batch_size=5000


ar_corpus_fname="UNv1.0.ar-en-clean2.ar"
en_corpus_fname="UNv1.0.ar-en.en"
cur_walign_dir="un-walign"

full_align_dir_path=os.path.join(cur_walign_dir,exp_dir)
if not os.path.exists(full_align_dir_path): os.makedirs(full_align_dir_path)


params={}
params["lang"]="en"
params["ignore_punc"]=True
params["stop_words"]=[]
params["ignore_ar_pre_suf"]=True
params["remove_al"]=True
params["index_words"]=True
params["lower"]=True
params["stemming"]=False

walign_params={}
walign_params["min_retrieved_index_count"]=3
walign_params["max_dist"]=5
walign_params["max_sent_size"]=60
walign_params["min_ratio"]=0.05
walign_params["n_epochs"]=5

walign_params_fpath=os.path.join(full_align_dir_path,"walign-params.json")
indexing_params_fpath=os.path.join(full_align_dir_path,"indexing-params.json")
with open(walign_params_fpath,"w") as walign_fopen:
  json.dump(walign_params,walign_fopen)


#print("file size:", file_len(en_corpus_fname))

src_params,trg_params=dict(params),dict(params) #we copy the default parameters
src_params["stop_words"]=["the","of","an","a","and","is","are", "to","been","it","as","for","have"]
trg_params["stop_words"]=["إلى"]

indexing_params={}
indexing_params["src"]=src_params
indexing_params["trg"]=trg_params
with open(indexing_params_fpath,"w") as indexing_fopen:
  json.dump(indexing_params,indexing_fopen)



ar_fopen=open(ar_corpus_fname)
en_fopen=open(en_corpus_fname)
src_sents,trg_sents=[],[]
for pair_i,pair in enumerate(zip(en_fopen,ar_fopen)):
  if pair_i>batch_size: break
  #print(pair_i)
  if pair_i>0 and pair_i%batch_size==0: #if the current line number of the bitext matches the btach size, we start processing batch
    #walign_sentences(src_sents,trg_sents,cur_align_fname)
    cur_align_fname=os.path.join(full_align_dir_path,"%s.txt"%pair_i) #the output goes into the filename named after the current line number
    cur_size=file_len(cur_align_fname) #we check how many lines in the current file
    if cur_size==len(src_sents): continue #if the files has an equal number of lines to the batch size, we proceed to next batch
    #uncheck the above line after you finish experiments
    print("indexing src sentences")
    
    src_index_obj=indexing(src_sents, tok,src_params)
    print("indexing trg sentences")
    trg_index_obj=indexing(trg_sents, cur_ar_tok_fn,trg_params)
    print("finished indexing, now aligning words")
    print("number of sentences:",len(src_sents))
    # start_i=1000
    # n_sents=50
    # sent_counter=0
    for sent_i in range(len(src_sents)): #range(start_i,start_i+n_sents):
      if sent_i%1000==0: print("now aligning", sent_i)
      #sent_counter+=1
      #if sent_counter>35: break
      
      if sent_i<cur_size: 
        #print("skipping now")
        continue #also uncheck this after finishing experiments
      print("sent_i",sent_i)
      #cur_obj={}
      #print(src_sents[sent_i])
      #print(trg_sents[sent_i])
      # cur_src0=src_sents[i_] 
      # cur_trg0=trg_sents[i_]
      cur_src_obj0=src_index_obj.sent_obj_list[sent_i]
      cur_trg_obj0=trg_index_obj.sent_obj_list[sent_i]
      output0=walign(cur_src_obj0,cur_trg_obj0,src_index_obj.inv_index,trg_index_obj.inv_index,walign_params0=walign_params)
      #walign(cur_src_obj0,cur_trg_obj0,src_index_obj.inv_index,trg_index_obj.inv_index)
      #cur_obj["src"]=src_sents[sent_i]
      json_str=json.dumps(output0)
      cur_align_fopen=open(cur_align_fname,"a")
      cur_align_fopen.write(json_str+"\n")
      cur_align_fopen.close()


    
    print(pair_i, "number of unique src words", len(src_index_obj.inv_index))
    print(pair_i, "number of unique trg words", len(trg_index_obj.inv_index))
    print("resetting sentence lists")
    print("----------")
    #print("processing sentences")
    src_sents,trg_sents=[],[]
    #break
  if pair_i%10000==0: print(pair_i)
  src_line,trg_line=pair
  src_line,trg_line=src_line.strip("\n\r\t "),trg_line.strip("\n\r\t ")
  src_sents.append(src_line)
  trg_sents.append(trg_line)

ar_fopen.close()
en_fopen.close()

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
indexing src sentences
indexing trg sentences
finished indexing, now aligning words
number of sentences: 100000
now aligning 0
sent_i 0
skipping now
sent_i 1
skipping now
sent_i 2
skipping now
sent_i 3
skipping now
sent_i 4
skipping now
sent_i 5
skipping now
sent_i 6
skipping now
sent_i 7
skipping now
sent_i 8
skipping now
sent_i 9
skipping now
sent_i 10
skipping now
sent_i 11
skipping now
sent_i 12
skipping now
sent_i 13
skipping now
sent_i 14
skipping now
sent_i 15
skipping now
sent_i 16
skipping now
sent_i 17
skipping now
sent_i 18
skipping now
sent_i 19
skipping now
sent_i 20
skipping now
sent_i 21
skipping now
sent_i 22
skipping now
sent_i 23
skipping now
sent_i 24
skipping now
sent_i 25
skipping now
sent_i 26
skipping now
sent_i 27
skipping now
sent_i 28
skipping now
sent_i 29
skipping now
sent_i 30
skipping now
sent_i 31
skipping now
sent_i 32
skipping now
sent_i 33
skipping now
sent_i 34
skipping now
sent_i 35
skipping now

##Testing Alignment Output

In [65]:
start_i=3900
n_sents=20
sent_counter=0
for sent_i in range(start_i,start_i+n_sents):
  #if sent_i%1000==0: print("now aligning", sent_i)
  sent_counter+=1
  #if sent_counter>35: break
  print("sent_i",sent_i)
  #if sent_i<cur_size: continue #also uncheck this after finishing experiments
  cur_obj={}
  print(src_sents[sent_i])
  print(trg_sents[sent_i])
  # cur_src0=src_sents[i_] 
  # cur_trg0=trg_sents[i_]
  cur_src_obj0=src_index_obj.sent_obj_list[sent_i]
  #if len(cur_src_obj0.tokens)>20: continue
  cur_trg_obj0=trg_index_obj.sent_obj_list[sent_i]
  output0=walign(cur_src_obj0,cur_trg_obj0,src_index_obj.inv_index,trg_index_obj.inv_index,walign_params0=walign_params)
  cur_aligned_spans=output0.get("align")
  if cur_aligned_spans!=None:
    cur_src_tokens=output0["src"]
    cur_trg_tokens=output0["trg"]
    for src_span0,trg_span0 in cur_aligned_spans:
      print("src_span0,trg_span0",src_span0,trg_span0)
      print(cur_src_tokens[src_span0[0]:src_span0[1]+1])
      print(cur_trg_tokens[trg_span0[0]:trg_span0[1]+1])
    # for a,b in output0.items():
    #   print(a,b)
    print("-------")
  #print(output0)
  # cur_obj["src"]=src_sents[sent_i]
  # json_str=json.dumps(cur_obj)
  # cur_align_fopen=open(cur_align_fname,"a")
  # cur_align_fopen.write(json_str+"\n")
  # cur_align_fopen.close()

sent_i 3900
For the period ending 30 April 1993, voluntary contributions received by UNIKOM for which no budgetary provision was made in the cost estimates, and as valued by the donor Government, are shown below. 93-40057 (E) 160793 /... Estimated value (United States
وفيما يتعلق بالفترة المنتهية في 30 نيسان/أبريل 1993، يرد أدناه بيان التبرعات التي تلقتها بعثة المراقبة والتي لم يرصد بشأنها اعتماد في الميزانية في تقدير التكاليف، ووفقا للقيمة التي حددتها الحكومة المتبرعة.
0 (((0, 23), (0, 32)), 7.862982656080205) cur_score 0 top 0
1 (((0, 26), (0, 32)), 8.597533636525494) cur_score 7.658684337986297 top 0
2 (((0, 35), (0, 34)), 9.134889846843263) cur_score 9.134889846843263 top 7.658684337986297
3 (((0, 35), (0, 34)), 9.501195559987828) cur_score 9.501195559987828 top 9.134889846843263
4 (((0, 35), (0, 34)), 9.521654623591965) cur_score 9.521654623591965 top 9.501195559987828
src_span0,trg_span0 (0, 0) (0, 0)
['<s>']
['<s>']
src_span0,trg_span0 (3, 3) (5, 5)
['period']
['ال_فترة']
src_sp

In [24]:
span1=(5,15)
span2=(18,19)
y_dist=get_span_dist(span1,span2)
print(y_dist)

3


In [ ]:
ar_corpus_fname="UNv1.0.ar-en-clean2.ar"
en_corpus_fname="UNv1.0.ar-en.en"

n_sents_max=200000
ar_fopen=open(ar_corpus_fname)
en_fopen=open(en_corpus_fname)

src_sents,trg_sents=[],[]
for i,pair in enumerate(zip(en_fopen,ar_fopen)):
  if i>=n_sents_max: break
  if i%20000==0: print(i)
  en_line,ar_line=pair
  en_line,ar_line=en_line.strip("\n\r\t "),ar_line.strip("\n\r\t ")
  src_sents.append(en_line)
  trg_sents.append(ar_line)

ar_fopen.close()
en_fopen.close()
print(len(src_sents),len(src_sents))
print("Now adding another bitext in a Google sheet")
import pandas as pd
#Job Bitexts
#xls_path='https://docs.google.com/spreadsheets/d/e/2PACX-1vSqipSUAaMWJrizvZtg1WIpds2pIiXtcZC6wjOLeVYie9p5O0lqiPnVdd8ambGKH8B0qJ4kmlVeEs6f/pub?output=xlsx'
#Ref bitexts
xls_path='https://docs.google.com/spreadsheets/d/e/2PACX-1vSVy7hY4fV0LHqTCypOjZAM9OrBevDDnpIcEKEAPKDsYRoxAz9TUBcDIgy8PvHkw5IDAZ6eRmKdZuRw/pub?output=xlsx'
cur_xls = pd.ExcelFile(xls_path)
for cur_sheet_name in cur_xls.sheet_names:
  print(cur_sheet_name)
  #cur_sheet_name="2"
  cur_sheet_obj=pd.read_excel(cur_xls, cur_sheet_name,keep_default_na=False)
  cur_sheet_obj['en']=cur_sheet_obj['en'].apply(str)
  cur_sheet_obj['ar']=cur_sheet_obj['ar'].apply(str)
  # print(cur_sheet_obj)
  # continue

  sent_pair_list=[]
  for i,row in enumerate(cur_sheet_obj.iterrows()):
    row_dict=row[1].to_dict()
    en0,ar0=row_dict["en"],row_dict["ar"]
    ar0=clean_ar(ar0)
    sent_pair_list.append((en0,ar0))
    src_sents.append(en0)
    trg_sents.append(ar0)

#Now indexing
print("Now indexing all sentences")
src_indexing_obj=indexing(src_sents,tok,stop_words=["the","of","and","its","his","her","it","is","are", "their"],ignore_punc=True)
print("finihsed indexing source sentences")
trg_indexing_obj=indexing(trg_sents,cur_ar_tok_fn,ignore_ar_pre_suf=True,ignore_punc=True)
print("finihsed indexing target sentences")

0
20000
40000
60000
80000
100000
120000
140000
160000
180000
200000 200000
Now adding another bitext in a Google sheet
A75760
A76562
A76564
A76572
A76529
A76547
A74594
A76549
A76552
A74597
A75785
Now indexing all sentences
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
finihsed indexing source sentences
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
finihsed indexing target sentences


#Starting Alignmenet

In [ ]:
import math

def overlaps(span1,span2):
  span1_i0,span1_i1=span1
  span2_i0,span2_i1=span2
  if span1_i0>=span2_i0 and span1_i0<=span2_i1: return True
  if span1_i1>=span2_i0 and span1_i1<=span2_i1: return True
  if span2_i0>=span1_i0 and span2_i0<=span1_i1: return True
  if span2_i1>=span1_i0 and span2_i1<=span1_i1: return True
  return False

def get_inner_dist(span1,span2):
  combined_xs=[]
  combined_xs.extend(span1)
  combined_xs.extend(span2)  
  combined_xs.sort()
  return combined_xs[2]-combined_xs[1]

def get_rec_el_children(el0,el_child_dict0,el_list0=[]):
  cur_children0=el_child_dict0.get(el0,[])
  for ch0 in cur_children0:
    el_list0.append(ch0)
    el_list0=get_rec_el_children(ch0,el_child_dict0,el_list0)
  return el_list0

def get_min_max(range1,range2):
  range1_x0,range1_x1=range1
  range2_x0,range2_x1=range2
  min_x=min(range1_x0,range1_x1,range2_x0,range2_x1)
  max_x=max(range1_x0,range1_x1,range2_x0,range2_x1)
  return (min_x,max_x)

def get_area_factor(x_range1,y_range1):
  x_dist=x_range1[1]-x_range1[0]+1
  y_dist=y_range1[1]-y_range1[0]+1
  cur_factor=(x_dist+y_dist)/2
  return cur_factor #math.log(cur_factor)


def get_tok_loc(tok_list):
  tok_locs_list=[]
  for i_,cur_tok in enumerate(tok_list):
    tok_locs_list.append(((cur_tok,),(i_,i_))) #to get unigrams and their locations
    if i_<len(tok_list)-1: tok_locs_list.append(((cur_tok,tok_list[i_+1]), (i_,i_+1))) #to get bigrams and their locations
  tok_locs_list.sort(key=lambda x:x[0])
  grouped=[(key,[v[1] for v in list(group)]) for key,group in groupby(tok_locs_list,lambda x:x[0])]
  return grouped


class walign:
  def __init__(self,src0,trg0,src_inv_dict0,trg_inv_dict0,max_n_elms=10,min_ratio=0.05):
    span_el_dict={} #which element corresponds to this span
    span_wt_dict={} #wight of each span
    span_child_dict={} #which subspans this span consists of
    
    span_top_elms_dict={} #show the top elements for each span
    el_wt_dict={} #show the weight of each element
    start_span_dict={} #show all the spans corresponding to a src location
    el_child_dict={} #show which elements are children of which
    all_phrase_pairs=[]

    src0=['<s>']+src0+['</s>']
    trg0=['<s>']+trg0+['</s>']
    self.src0=src0
    self.trg0=trg0
    src_grouped=get_tok_loc(src0) #group all the locations for a list of consecutive tokens (src then trg)
    trg_grouped=get_tok_loc(trg0)
    src_side,trg_side=[],[]

    for s_toks,s_locs_grp in src_grouped: #retrive the indexes for each token sequence, map them to the locations
      src_indexes0= rtrv_idx(s_toks,src_inv_dict0)
      src_side.append((s_toks,s_locs_grp,src_indexes0))
    for t_toks,t_locs_grp in trg_grouped:
      trg_indexes0= rtrv_idx(t_toks,trg_inv_dict0)
      trg_side.append((t_toks,t_locs_grp,trg_indexes0))

    #now starting the match the indexes of both src and trg token locations and get the corresponding rations/weights
    matching_list=[]
    all_elements=[] #to create elements (pairs of src-trg spans)
    tmp_ratio_dict={}
    for s_toks,s_locs_grp,src_indexes0 in src_side:
      s_toks_str=" ".join(s_toks)
      for t_toks,t_locs_grp,trg_indexes0 in trg_side:
        cur_ratio=0
        t_toks_str=" ".join(t_toks)
        if s_toks==t_toks: cur_ratio=1
        else: 
          if len(src_indexes0)==0 or len(trg_indexes0)==0: continue #cur_ratio= 0
          cur_ratio=idx_match(src_indexes0,trg_indexes0,5)
        if cur_ratio==0: continue
        for sl0 in s_locs_grp:
          for tl0 in t_locs_grp:
            cur_el=(sl0,tl0) #an element is a tuple containing the span of src indexes and span of trg indexes
            tmp_ratio_dict[cur_el]=cur_ratio
            #all_elements.append((cur_el,s_toks_str,t_toks_str,cur_ratio))
            all_elements.append((cur_el,cur_ratio))
            found_wt=span_wt_dict.get(sl0,0)
            if found_wt<cur_ratio:
              span_wt_dict[sl0]=cur_ratio
              span_el_dict[sl0]=cur_el
              span_child_dict[sl0]=[]
    #####
    #and now let's group all the elements by the starting location of the src tokenized sentence
    all_elements.sort(key=lambda x:(x[0][0],-x[-1]))
    all_elements_grouped=[(key,list(group)) for key,group in groupby(all_elements,lambda x:x[0][0])]

    #initially populating the start/span/element/weight dictionaries
    for k0,grp0 in all_elements_grouped:
      for el0,el_wt0 in grp0[:max_n_elms]:
        if el_wt0<min_ratio: continue
        el_wt_dict[el0]=el_wt0

    all_elements.sort(key=lambda x:-x[-1])
    used_src,used_trg=[],[]
    new_el_wt_list=[]
    for ae in all_elements: #get the elements below the top by iteratively discarding the top used elements
      el0,wt0=ae
      src_span,trg_span=el0    
      if any([v in used_src for v in src_span]): continue
      if any([v in used_trg for v in trg_span]): continue
      used_src.extend(src_span)
      used_trg.extend(trg_span)
      new_el_wt_list.append(ae)
      el_wt_dict[el0]=wt0#+0.2
    
    #all_elements.sort(key=lambda x:(x[0][0][0],-x[-1]))

    el_wt_list=list(el_wt_dict.items())
    el_wt_list.sort(key=lambda x:(x[0][0],-x[-1]))
    for el0,wt0 in el_wt_list:
      #print(el0,wt0)
      src_span0,trg_span0=el0
      start_i=src_span0[0]
      found_spans=start_span_dict.get(start_i,[])
      if not src_span0 in found_spans: start_span_dict[start_i]=found_spans+[src_span0]
      found_elements=span_top_elms_dict.get(src_span0,[])
      if not el0 in found_elements: span_top_elms_dict[src_span0]=found_elements+[el0]


    used_el_dict={}
    full_src_span=(0,len(src0)-1)
    full_trg_span=(0,len(trg0)-1)
    src_trg_span=(full_src_span,full_trg_span)
    #cur_full_span_wt=span_wt_dict.get(full_src_span,0)
    cur_el_full_span_wt=el_wt_dict.get(src_trg_span,0) #check if there is an element that covers the full src/trg span
    

    for inc0 in range(15): #increments to cover slices of src location indexes
      
      test_el_wt=sorted(list(el_wt_dict.items()),key=lambda x:-x[-1])
      tmp_el_full_span_wt=el_wt_dict.get(src_trg_span,0)
      print(inc0,test_el_wt[0], tmp_el_full_span_wt) #Checking the situation with the convergence of the full src/trg span element
      if tmp_el_full_span_wt>0:
        if tmp_el_full_span_wt>cur_el_full_span_wt: cur_el_full_span_wt=tmp_el_full_span_wt
        else: 
          print("alignment solved")
          break


      for i0 in range(len(src0)):
        first_spans=start_span_dict.get(i0,[])
        first_elements=[]
        for sp0 in first_spans:
          first_elements.extend(span_top_elms_dict.get(sp0,[]))
        valid_next_items_counter=0
        for el0 in first_elements:
          el0_wt=el_wt_dict.get(el0,0)
          el0_src_span,el0_trg_span=el0
          el0_src_span_i0,el0_src_span_i1=el0_src_span
          el0_trg_span_i0,el0_trg_span_i1=el0_trg_span

          second_spans=[]
          second_elements=[]
          for i1 in range(el0_src_span_i1,el0_src_span_i1+5):
            second_spans.extend(start_span_dict.get(i1,[]))
          for sp1 in second_spans: second_elements.extend(span_top_elms_dict.get(sp1,[]))
          for el1 in second_elements:
            if el1==el0: continue #we should clean up all the skipping conditions to make sure everything is organized and accounted for, and we're not skipping something that should be kept
            el1_wt=el_wt_dict.get(el1,0)
            el1_src_span,el1_trg_span=el1
            el1_src_span_i0,el1_src_span_i1=el1_src_span
            el1_trg_span_i0,el1_trg_span_i1=el1_trg_span
            trg_spans_overlap=overlaps(el0_trg_span,el1_trg_span)
            src_spans_overlap=overlaps(el0_src_span,el1_src_span)
            
            if el0_src_span==el1_src_span and el0_src_span_i0!=el0_src_span_i1: continue #if [(1,2),(4,5)] [(1,2),(7,6)]
            if trg_spans_overlap: continue #skip if target spans overlap
            if src_spans_overlap and el0_src_span!=el1_src_span: continue

            trg_span_inner_dist=get_inner_dist(el0_trg_span,el1_trg_span)
            if trg_span_inner_dist>10: continue #skip if there is much distance between target spans

            src_span_inner_dist=get_inner_dist(el0_src_span,el1_src_span)
            
            delta_x=el0_src_span_i1-el0_src_span_i0
            excluded=False
            trg_outer_dist=max(el1_trg_span) - min(el0_trg_span)
            if el0_src_span==el1_src_span:
              if el1_trg_span_i1<el0_trg_span_i0: excluded=True
              elif trg_outer_dist>6: excluded=True
              elif trg_span_inner_dist>1: excluded=True
            
            combined_src_span=get_min_max(el0_src_span,el1_src_span)
            combined_trg_span=get_min_max(el0_trg_span,el1_trg_span) 
            if combined_src_span[1]-combined_src_span[0]<2:
              if combined_trg_span[1]-combined_trg_span[0]>8: excluded=True
            if combined_src_span[1]-combined_src_span[0]<3:
              if combined_trg_span[1]-combined_trg_span[0]>10: excluded=True
            

            el_pair=(el0,el1)
            if used_el_dict.get(el_pair,False)==True: excluded=True
            used_el_dict[el_pair]=True

            if excluded: continue

            
            combined_wt=el0_wt+el1_wt
            

            area_factor=get_area_factor(combined_src_span,combined_trg_span)
            #combined_wt=combined_wt-0.02*area_factor
            combined_wt=combined_wt-trg_span_inner_dist*0.05

            combined_el=(combined_src_span,combined_trg_span)
            found_wt=el_wt_dict.get(combined_el,0)
            if found_wt>0 and (combined_src_span[0]==combined_src_span[1]): continue #very sketechy solution to avoid situations where a bigram is replaced by a less likely combination of two consecutive unigrams
            if found_wt>0 and (combined_trg_span[0]==combined_trg_span[1]): continue
            if combined_wt>found_wt:
              el_wt_dict[combined_el]=combined_wt
              el_child_dict[combined_el]=(el0,el1)
              tmp_elements_in_span=span_top_elms_dict.get(combined_src_span,[])
              if not combined_el in tmp_elements_in_span:
                tmp_elements_in_span.append(combined_el)
                tmp_elements_in_span.sort(key=lambda x:-el_wt_dict.get(x,0))
                span_top_elms_dict[combined_src_span]=tmp_elements_in_span[:max_n_elms]
              tmp_spans_starting_with_i0=start_span_dict.get(i0,[])
              if not combined_src_span in tmp_spans_starting_with_i0:
                start_span_dict[i0]=tmp_spans_starting_with_i0+[combined_src_span]
            #if found_wt>0:
            # if el0_src_span==el1_src_span and False:
            #   print(inc0,"excluded",excluded,trg_outer_dist)
            #   print(el0,el0_wt)
            #   print(el1,el1_wt)
            #   print("combined:",combined_el,combined_wt)
            #   print("found:",found_wt)
            #   print("-----")

            valid_next_items_counter+=1

    all_elements2=get_rec_el_children(src_trg_span,el_child_dict,el_list0=[])
    #all_elements=get_rec_children(list_spans_wts[0][0],span_child_dict,span_el_dict,el_list0=[])
    self.final_el_list=[]
    for ch_el in all_elements2:
      #print(ch_el, el_wt_dict[ch_el])
      self.final_el_list.append((ch_el, el_wt_dict[ch_el]))
      src_span0,trg_span0=ch_el
      cur_src_str=" ".join([src0[i_] for i_ in range(src_span0[0],src_span0[1]+1)])
      cur_trg_str=" ".join([trg0[i_] for i_ in range(trg_span0[0],trg_span0[1]+1)])  
      cur_pair=(cur_src_str,cur_trg_str)
      # print(cur_src_str)   
      # print(cur_trg_str) 
      # print("--") 
      # all_phrase_pairs.append((cur_pair))
      #all_phrase_pairs.append((cur_pair,sent_counter))
    # for a in all_phrase_pairs:
    #   print(a)

src_list=src_indexing_obj.all_tok_sentences
trg_list=trg_indexing_obj.all_tok_sentences
src_list_orig=src_indexing_obj.all_tok_original_sentences
trg_list_orig=trg_indexing_obj.all_tok_original_sentences
src_mappings_list=src_indexing_obj.all_mappings
trg_mappings_list=trg_indexing_obj.all_mappings


src_inv_dict=src_indexing_obj.inv_index
trg_inv_dict=trg_indexing_obj.inv_index

phrase_fpath="phrase_list_test.txt"
fopen=open(phrase_fpath,"w")

test_i=7104
n_sents=1
sent_counter=0
all_phrase_pairs=[]
for test_i in range(0,1000):
  print(test_i)
  src_tokens,trg_tokens=src_list[test_i],trg_list[test_i]
  src_mappings,trg_mappings=src_mappings_list[test_i],trg_mappings_list[test_i]
  src_original_tokens,trg_original_tokens=src_list_orig[test_i],trg_list_orig[test_i]
  # print("src_mappings",src_mappings)
  # print("trg_mappings",trg_mappings)
  src_original_tokens=["<s>"]+src_original_tokens+["</s>"]
  trg_original_tokens=["<s>"]+trg_original_tokens+["</s>"]

  new_src_mappings=[0]+[v+1 for v in src_mappings]+[len(src_original_tokens)-1]
  new_trg_mappings=[0]+[v+1 for v in trg_mappings]+[len(trg_original_tokens)-1]
  # print("new_src_mappings",new_src_mappings)
  # print("new_trg_mappings",new_trg_mappings)
  align_obj=walign(src_tokens,trg_tokens,src_inv_dict,trg_inv_dict,max_n_elms=10)
  # print(src_tokens)
  # print(trg_tokens)

  # print("Original SRC: >>>",src_original_tokens)
  # print("Original TRG: >>>",trg_original_tokens)
  # print("orig src:",len(src_original_tokens), "orig trg:",len(trg_original_tokens))
  # print("filtered src:",len(src_tokens), "filtered trg:",len(trg_tokens))
  # print("aligned src:",len(align_obj.src0), "aligned trg:",len(align_obj.trg0))
  for el0,wt0 in align_obj.final_el_list:
    # print(el0,wt0)
    src_span0,trg_span0=el0
    src_i0,src_i1=src_span0
    trg_i0,trg_i1=trg_span0
    new_src_i0,new_src_i1=new_src_mappings[src_i0],new_src_mappings[src_i1]
    new_trg_i0,new_trg_i1=new_trg_mappings[trg_i0],new_trg_mappings[trg_i1]
    src_phrase_tokens=src_original_tokens[new_src_i0:new_src_i1+1]
    trg_phrase_tokens=trg_original_tokens[new_trg_i0:new_trg_i1+1]
    src_phrase_str=" ".join(src_phrase_tokens)
    trg_phrase_str=" ".join(trg_phrase_tokens)
    # print(src_phrase_str)
    # print(trg_phrase_str)
    line='%s\t%s\t%s\n'%(src_phrase_str,trg_phrase_str,round(wt0,4))
    fopen.write(line)
    # #src_span0,trg_span0=ch_el
    # cur_src_str=" ".join([src_tokens[i_] for i_ in range(src_span0[0],src_span0[1]+1)])
    # cur_trg_str=" ".join([trg_tokens[i_] for i_ in range(trg_span0[0],trg_span0[1]+1)])  
    # print("Filtered - SRC: >>>",cur_src_str)
    # print("Filtered - TRG: >>>",cur_trg_str)

  # print("----")

fopen.close()
# for src_tokens,trg_tokens in zip(src_list[test_i:test_i+n_sents],trg_list[test_i:test_i+n_sents]):
#   print(sent_counter)
#   print(src_tokens)
#   print(trg_tokens)
  

Streaming output truncated to the last 5000 lines.
1 (((12, 14), (10, 14)), 2.021784913973186) 0
2 (((0, 13), (0, 11)), 3.0805376007702883) 0
3 (((0, 14), (0, 14)), 5.522061867548715) 5.522061867548715
4 (((0, 14), (0, 14)), 5.522061867548715) 5.522061867548715
alignment solved
337
0 (((0, 0), (0, 0)), 1) 0
1 (((0, 4), (0, 6)), 1.613978494623656) 0
2 (((0, 7), (0, 6)), 1.6272819883449043) 0
3 (((0, 21), (0, 18)), 2.581283214889811) 0
4 (((0, 22), (0, 18)), 2.816168752231888) 0
5 (((0, 22), (0, 18)), 2.816168752231888) 0
6 (((0, 22), (0, 18)), 2.816168752231888) 0
7 (((0, 22), (0, 18)), 2.816168752231888) 0
8 (((0, 22), (0, 18)), 2.816168752231888) 0
9 (((0, 22), (0, 18)), 2.816168752231888) 0
10 (((0, 22), (0, 18)), 2.816168752231888) 0
11 (((0, 22), (0, 18)), 2.816168752231888) 0
12 (((0, 22), (0, 18)), 2.816168752231888) 0
13 (((0, 22), (0, 18)), 2.816168752231888) 0
14 (((0, 22), (0, 18)), 2.816168752231888) 0
338
0 (((0, 0), (0, 0)), 1) 0
1 (((44, 46), (40, 42)), 2.2500868325172014

#Processing Phrase Table

In [ ]:
phrase_fpath="phrase_list_test.txt"
phrase_fopen=open(phrase_fpath)
phrase_corr_dict={}
for i0,line0 in enumerate(phrase_fopen):
  #continue
  #if i0>5000: break
  line_split=line0.strip().split("\t")
  if len(line_split)<3: continue
  src0,trg0,wt0=line_split[:3]
  wt0=float(wt0)
  if src0==trg0: continue
  src0_lower=src0.lower()
  tmp_dict=phrase_corr_dict.get(src0_lower,{})
  found_total_count,found_total_wt=tmp_dict.get(trg0,(0,0))
  found_total_count,found_total_wt=found_total_count+1,found_total_wt+wt0
  tmp_dict[trg0]=(found_total_count,found_total_wt)
  phrase_corr_dict[src0_lower]=tmp_dict




  #print([src0,trg0,wt0])

phrase_fopen.close()
for a,b in phrase_corr_dict.items():
  for a1,b1 in b.items():
    if b1[0]<5: continue
    print(a)
    print(a1)
    print(b1)
    print("-----")


international law commission
لجنة ال_قانون ال_دولي
(5, 6.6690000000000005)
-----
international law
ال_قانون ال_دولي
(10, 11.372)
-----
international
ال_دولي
(31, 14.904800000000007)
-----
international
دولي
(21, 10.096800000000004)
-----
international
دولية
(34, 13.739399999999993)
-----
international
ال_دولية
(35, 14.143499999999992)
-----
law
ال_قانون
(43, 30.375199999999978)
-----
law
قانون
(5, 3.532)
-----
commission
لجنة
(9, 2.2184999999999997)
-----
commission
ال_لجنة
(20, 4.9300000000000015)
-----
forty
ال_أربعين
(13, 7.695999999999998)
-----
session
ال_دورة
(10, 5.198999999999999)
-----
session
دورتها
(9, 3.1032)
-----
may
أيار / مايو
(10, 3.5210000000000004)
-----
may
قد
(11, 0.8162000000000001)
-----
july
تموز / يوليه
(10, 9.006)
-----
comments
تعليقات
(11, 8.3897)
-----
comments
ال_تعليقات
(7, 5.3389)
-----
governments
ال_حكومات
(22, 17.48119999999999)
-----
on
على
(41, 9.753899999999993)
-----
on
بشأن
(13, 2.1476)
-----
on
في
(18, 4.433400000000001)
-----
on
من
(11, 2.02840

In [ ]:
import math
src_list=src_indexing_obj.all_tok_sentences
trg_list=trg_indexing_obj.all_tok_sentences
src_inv_dict=src_indexing_obj.inv_index
trg_inv_dict=trg_indexing_obj.inv_index

def overlaps(span1,span2):
  span1_i0,span1_i1=span1
  span2_i0,span2_i1=span2
  if span1_i0>=span2_i0 and span1_i0<=span2_i1: return True
  if span1_i1>=span2_i0 and span1_i1<=span2_i1: return True
  if span2_i0>=span1_i0 and span2_i0<=span1_i1: return True
  if span2_i1>=span1_i0 and span2_i1<=span1_i1: return True
  return False

def get_inner_dist(span1,span2):
  combined_xs=[]
  combined_xs.extend(span1)
  combined_xs.extend(span2)  
  combined_xs.sort()
  return combined_xs[2]-combined_xs[1]

# def get_rec_children(span0,span_child_dict0,span_el_dict0,el_list0=[]):
#   cur_children0=span_child_dict0.get(span0,[])
#   for ch0 in cur_children0:
#     el_list0.append(span_el_dict0[ch0])
#     el_list0=get_rec_children(ch0,span_child_dict0,span_el_dict0,el_list0)
#   return el_list0

def get_rec_el_children(el0,el_child_dict0,el_list0=[]):
  cur_children0=el_child_dict0.get(el0,[])
  for ch0 in cur_children0:
    el_list0.append(ch0)
    el_list0=get_rec_el_children(ch0,el_child_dict0,el_list0)
  return el_list0

def get_min_max(range1,range2):
  range1_x0,range1_x1=range1
  range2_x0,range2_x1=range2
  min_x=min(range1_x0,range1_x1,range2_x0,range2_x1)
  max_x=max(range1_x0,range1_x1,range2_x0,range2_x1)
  return (min_x,max_x)

def get_dist_factor(el1,el2): #[(5,6),(9,9)] - [(7,7),(12,13)] #get inner distance between two elements, 1 if adjacent
  el1_xrange,el1_yrange=el1
  el2_xrange,el2_yrange=el2
  combined_xs=[]
  combined_xs.extend(el1_xrange)
  combined_xs.extend(el2_xrange)
  combined_ys=[]
  combined_ys.extend(el1_yrange)
  combined_ys.extend(el2_yrange)
  combined_xs.sort()
  combined_ys.sort()
  inner_xdist=combined_xs[2]-combined_xs[1]
  inner_ydist=combined_ys[2]-combined_ys[1]
  return (inner_xdist+inner_ydist)/2

def get_area_factor(x_range1,y_range1):
  x_dist=x_range1[1]-x_range1[0]+1
  y_dist=y_range1[1]-y_range1[0]+1
  cur_factor=(x_dist+y_dist)/2
  return cur_factor #math.log(cur_factor)


def get_tok_loc(tok_list):
  tok_locs_list=[]
  for i_,cur_tok in enumerate(tok_list):
    tok_locs_list.append(((cur_tok,),(i_,i_)))
    if i_<len(tok_list)-1: tok_locs_list.append(((cur_tok,tok_list[i_+1]), (i_,i_+1)))
  tok_locs_list.sort(key=lambda x:x[0])
  grouped=[(key,[v[1] for v in list(group)]) for key,group in groupby(tok_locs_list,lambda x:x[0])]
  return grouped


test_i=-len(sent_pair_list)
n_sents=len(sent_pair_list)-2

max_n_elms=10 #max number of elements per span

test_i=4001
test_i=100
#test_i=12005
n_sents=1
sent_counter=0
all_phrase_pairs=[]
for src0,trg0 in zip(src_list[test_i:test_i+n_sents],trg_list[test_i:test_i+n_sents]):
  print(sent_counter)
  print(src0)
  print(trg0)
  span_el_dict={} #which element corresponds to this span
  span_wt_dict={} #wight of each span
  span_child_dict={} #which subspans this span consists of
  
  span_top_elms_dict={} #show the top elements for each span
  el_wt_dict={} #show the weight of each element
  start_span_dict={} #show all the spans corresponding to a src location
  el_child_dict={} #show which elements are children of which

  src0=['<s>']+src0+['</s>']
  trg0=['<s>']+trg0+['</s>']
  src_grouped=get_tok_loc(src0) #group all the locations for a list of consecutive tokens (src then trg)
  trg_grouped=get_tok_loc(trg0)
  src_side,trg_side=[],[]

  for s_toks,s_locs_grp in src_grouped: #retrive the indexes for each token sequence, map them to the locations
    src_indexes0= rtrv_idx(s_toks,src_inv_dict)
    src_side.append((s_toks,s_locs_grp,src_indexes0))
  for t_toks,t_locs_grp in trg_grouped:
    trg_indexes0= rtrv_idx(t_toks,trg_inv_dict)
    trg_side.append((t_toks,t_locs_grp,trg_indexes0))

  #now starting the match the indexes of both src and trg token locations
  matching_list=[]
  all_elements=[] #to create elements (pairs of src-trg spans)
  tmp_ratio_dict={}
  for s_toks,s_locs_grp,src_indexes0 in src_side:
    s_toks_str=" ".join(s_toks)
    #continue
    #print(s_toks,s_locs_grp,len(src_indexes0))
    for t_toks,t_locs_grp,trg_indexes0 in trg_side:
      cur_ratio=0
      t_toks_str=" ".join(t_toks)
      if s_toks==t_toks: cur_ratio=1
      else: 
        #cur_ratio=idx_match(src_indexes0,trg_indexes0)
        if len(src_indexes0)==0 or len(trg_indexes0)==0: continue #cur_ratio= 0
        cur_ratio=idx_match(src_indexes0,trg_indexes0,5)
      if cur_ratio==0: continue
      for sl0 in s_locs_grp:
        for tl0 in t_locs_grp:
          cur_el=(sl0,tl0) #an element is a tuple containing the span of src indexes and span of trg indexes
          tmp_ratio_dict[cur_el]=cur_ratio
          #all_elements.append((cur_el,s_toks_str,t_toks_str,cur_ratio))
          all_elements.append((cur_el,cur_ratio))
          found_wt=span_wt_dict.get(sl0,0)
          if found_wt<cur_ratio:
            span_wt_dict[sl0]=cur_ratio
            span_el_dict[sl0]=cur_el
            span_child_dict[sl0]=[]
  


      #matching_list.append((s_toks,t_toks,cur_ratio))
  #and now let's group all the elements by the starting location of the src tokenized sentence
  all_elements.sort(key=lambda x:(x[0][0],-x[-1]))
  all_elements_grouped=[(key,list(group)) for key,group in groupby(all_elements,lambda x:x[0][0])]

  #initially populating the start/span/element/weight dictionaries
  for k0,grp0 in all_elements_grouped:
    start_i=k0[0]
    #start_span_dict[start_i]=start_span_dict.get(start_i,[])+[k0]
    #span_top_elms_dict[k0]=[v[0] for v in grp0[:max_n_elms]]
    for el0,el_wt0 in grp0[:max_n_elms]:
      if el_wt0<0.01: continue
      el_wt_dict[el0]=el_wt0
      #print(g0)
  #   print("---")
  # for a,b in start_span_dict.items():
  #   print(a,b)
  all_elements.sort(key=lambda x:-x[-1])
  used_src,used_trg=[],[]
  new_el_wt_list=[]
  for ae in all_elements: #get the elements below the top by iteratively discarding the top used elements
    el0,wt0=ae
    src_span,trg_span=el0    
    if any([v in used_src for v in src_span]): continue
    if any([v in used_trg for v in trg_span]): continue
    used_src.extend(src_span)
    used_trg.extend(trg_span)
    new_el_wt_list.append(ae)
    el_wt_dict[el0]=wt0#+0.2
    #print(ae)

  
  all_elements.sort(key=lambda x:(x[0][0][0],-x[-1]))
  # all_elements=new_el_wt_list+all_elements
  # for ae0 in all_elements[:30]:
  #   print(">>", ae0)
  el_wt_list=list(el_wt_dict.items())
  el_wt_list.sort(key=lambda x:(x[0][0],-x[-1]))
  for el0,wt0 in el_wt_list:
    #print(el0,wt0)
    src_span0,trg_span0=el0
    start_i=src_span0[0]
    found_spans=start_span_dict.get(start_i,[])
    if not src_span0 in found_spans: start_span_dict[start_i]=found_spans+[src_span0]
    found_elements=span_top_elms_dict.get(src_span0,[])
    if not el0 in found_elements: span_top_elms_dict[src_span0]=found_elements+[el0]

    
  # for a,b in start_span_dict.items():
  #   print("start span >>>",a,b)
  # for a,b in span_top_elms_dict.items():
  #   print("top elements +++",a,b)

  # break
  # all_elements_grouped=[(key,list(group)) for key,group in groupby(all_elements,lambda x:x[0][0][0])]
  # el_start_dict={}
  # for k0,el_grp in all_elements_grouped:
  #   #el_start_dict[k0]=el_grp[:10] 
  #   el_start_dict[k0]=dict(iter(el_grp[:10])) #for each starting src index , we identify the top 10 elements that start with it
  used_el_dict={}
  full_src_span=(0,len(src0)-1)
  full_trg_span=(0,len(trg0)-1)
  src_trg_span=(full_src_span,full_trg_span)
  cur_full_span_wt=span_wt_dict.get(full_src_span,0)
  cur_el_full_span_wt=el_wt_dict.get(src_trg_span,0)
  
  #continue  
  for inc0 in range(15): #increments to cover slices of src location indexes
    
    # test_wt=sorted(list(span_wt_dict.items()),key=lambda x:-x[-1])
    # tmp_full_span_wt=span_wt_dict.get(full_src_span,0)
    # test_wt1=[v for v in test_wt if v[0][0]==0]
    #print(inc0,test_wt[0],test_wt1[0],full_src_span, tmp_full_span_wt)
    test_el_wt=sorted(list(el_wt_dict.items()),key=lambda x:-x[-1])
    #test_el_wt1=[v for v in test_el_wt if v[0][0]==0]
    tmp_el_full_span_wt=el_wt_dict.get(src_trg_span,0)
    print(inc0,test_el_wt[0], tmp_el_full_span_wt)
    if tmp_el_full_span_wt>0:
      if tmp_el_full_span_wt>cur_el_full_span_wt: cur_el_full_span_wt=tmp_el_full_span_wt
      else: 
        print("alignment solved")
        break

    


    for i0 in range(len(src0)):
      i1=i0+inc0
      if i1>len(src0)-1: continue
      #if inc0>15: continue
      #first_items=el_start_dict.get(i0,[])
      #first_items=el_start_dict.get(i0,{})

      #New approach
      first_spans=start_span_dict.get(i0,[])
      first_elements=[]
      for sp0 in first_spans:
        first_elements.extend(span_top_elms_dict.get(sp0,[]))
      # print(i0)
      # print("first_spans",len(first_spans),first_spans[:5])
      # print("first_elements",len(first_elements),first_elements[:5])
      valid_next_items_counter=0
      for el0 in first_elements:
        el0_wt=el_wt_dict.get(el0,0)
        el0_src_span,el0_trg_span=el0
        el0_src_span_i0,el0_src_span_i1=el0_src_span
        el0_trg_span_i0,el0_trg_span_i1=el0_trg_span

        second_spans=[]
        second_elements=[]
        for i1 in range(el0_src_span_i1,el0_src_span_i1+5):
          second_spans.extend(start_span_dict.get(i1,[]))
        for sp1 in second_spans: second_elements.extend(span_top_elms_dict.get(sp1,[]))
        for el1 in second_elements:
          if el1==el0: continue
          el1_wt=el_wt_dict.get(el1,0)
          el1_src_span,el1_trg_span=el1
          el1_src_span_i0,el1_src_span_i1=el1_src_span
          el1_trg_span_i0,el1_trg_span_i1=el1_trg_span
          trg_spans_overlap=overlaps(el0_trg_span,el1_trg_span)
          src_spans_overlap=overlaps(el0_src_span,el1_src_span)
          if el0_src_span==el1_src_span and el0_src_span_i0!=el0_src_span_i1: continue #if [(1,2),(4,5)] [(1,2),(7,6)]
          #
          if trg_spans_overlap: continue #skip if target spans overlap
          if src_spans_overlap and el0_src_span!=el1_src_span: continue

          trg_span_inner_dist=get_inner_dist(el0_trg_span,el1_trg_span)
          if trg_span_inner_dist>10: continue #skip if there is much distance between target spans

          src_span_inner_dist=get_inner_dist(el0_src_span,el1_src_span)
          
          #if el0_src_span_i0==el0_src_span_i1==el1_src_span_i0 and el0_src_span_i1!=el1_src_span_i0: continue #vertical element with non-vertical element
          #if el0_src_span_i0==el0_src_span_i1==el1_src_span_i1 and el0_src_span_i1!=el1_src_span_i0: continue #vertical element with non-vertical element
          #print(el0,el1)
          delta_x=el0_src_span_i1-el0_src_span_i0
          excluded=False
          trg_outer_dist=max(el1_trg_span) - min(el0_trg_span)
          if el0_src_span==el1_src_span:
            if el1_trg_span_i1<el0_trg_span_i0: excluded=True
            elif trg_outer_dist>6: excluded=True
            elif trg_span_inner_dist>1: excluded=True
          #if delta_x<2 and trg_outer_dist>8: excluded=True
          
          combined_src_span=get_min_max(el0_src_span,el1_src_span)
          combined_trg_span=get_min_max(el0_trg_span,el1_trg_span) 
          if combined_src_span[1]-combined_src_span[0]<2:
            if combined_trg_span[1]-combined_trg_span[0]>8: excluded=True
          if combined_src_span[1]-combined_src_span[0]<3:
            if combined_trg_span[1]-combined_trg_span[0]>10: excluded=True
          

          el_pair=(el0,el1)
          if used_el_dict.get(el_pair,False)==True: excluded=True
          used_el_dict[el_pair]=True

          if excluded: continue

          
          combined_wt=el0_wt+el1_wt
          

          area_factor=get_area_factor(combined_src_span,combined_trg_span)
          #combined_wt=combined_wt-0.02*area_factor
          combined_wt=combined_wt-trg_span_inner_dist*0.05

          combined_el=(combined_src_span,combined_trg_span)
          found_wt=el_wt_dict.get(combined_el,0)
          if found_wt>0 and (combined_src_span[0]==combined_src_span[1]): continue
          if found_wt>0 and (combined_trg_span[0]==combined_trg_span[1]): continue
          if combined_wt>found_wt:
            el_wt_dict[combined_el]=combined_wt
            el_child_dict[combined_el]=(el0,el1)
            tmp_elements_in_span=span_top_elms_dict.get(combined_src_span,[])
            if not combined_el in tmp_elements_in_span:
              tmp_elements_in_span.append(combined_el)
              tmp_elements_in_span.sort(key=lambda x:-el_wt_dict.get(x,0))
              span_top_elms_dict[combined_src_span]=tmp_elements_in_span[:max_n_elms]
            tmp_spans_starting_with_i0=start_span_dict.get(i0,[])
            if not combined_src_span in tmp_spans_starting_with_i0:
              start_span_dict[i0]=tmp_spans_starting_with_i0+[combined_src_span]
          #if found_wt>0:
          if el0_src_span==el1_src_span and False:
            print(inc0,"excluded",excluded,trg_outer_dist)
            print(el0,el0_wt)
            print(el1,el1_wt)
            print("combined:",combined_el,combined_wt)
            print("found:",found_wt)
            print("-----")

          valid_next_items_counter+=1
          #if valid_next_items_counter>10: continue

        #print("second_elements",len(second_elements),second_elements[:5])
          

      
      continue

  all_elements2=get_rec_el_children(src_trg_span,el_child_dict,el_list0=[])
  #all_elements=get_rec_children(list_spans_wts[0][0],span_child_dict,span_el_dict,el_list0=[])
  for ch_el in all_elements2:
    print(ch_el)
    src_span0,trg_span0=ch_el
    cur_src_str=" ".join([src0[i_] for i_ in range(src_span0[0],src_span0[1]+1)])
    cur_trg_str=" ".join([trg0[i_] for i_ in range(trg_span0[0],trg_span0[1]+1)])  
    cur_pair=(cur_src_str,cur_trg_str)
    print(cur_src_str)   
    print(cur_trg_str) 
    print("--") 
    all_phrase_pairs.append((cur_pair,sent_counter))
  sent_counter+=1
  # print("\t".join(src0))
  # for t0 in trg0: print(t0)

# for a in all_phrase_pairs:
#   print(a)




0
['this', 'a', 'further', 'aspect', 'broader', 'task', 'settling', 'relationship', 'between', 'statute', 'existing', 'treaties']
['هذا', 'جانب', 'آخر', 'من', 'مهمة', 'أعم', 'متمثلة', 'في', 'تسوية', 'علاقة', 'بين', 'نظام', 'أساسي', 'معاهدات', 'قائمة']
0 (((0, 0), (0, 0)), 1) 0
1 (((11, 13), (14, 16)), 1.8009590917987863) 0
2 (((6, 13), (5, 16)), 3.6476920594577056) 2.66096910074624
3 (((0, 13), (0, 16)), 5.257242927554253) 5.257242927554253
4 (((0, 13), (0, 16)), 5.257242927554253) 5.257242927554253
alignment solved
((0, 9), (0, 11))
<s> this a further aspect broader task settling relationship between
<s> هذا جانب آخر من مهمة أعم متمثلة في تسوية علاقة بين
--
((0, 6), (0, 6))
<s> this a further aspect broader task
<s> هذا جانب آخر من مهمة أعم
--
((0, 1), (0, 1))
<s> this
<s> هذا
--
((0, 0), (0, 0))
<s>
<s>
--
((1, 1), (1, 1))
this
هذا
--
((2, 6), (2, 6))
a further aspect broader task
جانب آخر من مهمة أعم
--
((2, 4), (2, 4))
a further aspect
جانب آخر من
--
((2, 3), (3, 4))
a further
آخر 

In [ ]:
cur_span=(47, 47)
cur_span=(21, 22)
cur_span=(21, 21)
#cur_span=(22, 22)
#cur_span=(4, 5)
#cur_span=(34, 35)
#cur_span=(31, 31)
#cur_span=(28, 30)
#cur_span=(29, 29)
#cur_span=(29, 31)
#cur_span=(35, 35)
#cur_span=(21, 22)
cur_span=(23, 23)
test=span_top_elms_dict.get(cur_span,[]) 
for a in test:
  print(a, el_wt_dict.get(a,0))
  src_str=" ".join(src0[a[0][0]:a[0][1]+1])
  trg_str=" ".join(trg0[a[1][0]:a[1][1]+1])
  print(src_str)
  print(trg_str)
  print("----")  
# ((21, 22), (21, 22)) 0.9872094270568594
# secretary general
# أمين عام
# ((21, 21), (21, 21)) 0.911346964257843
# secretary
# أمين
# ----
# ((21, 21), (21, 22)) 1.4313109930348213
# secretary
# أمين عام

((23, 23), (11, 11)) 0.447362499598187
to
في
----
((23, 23), (14, 14)) 0.447362499598187
to
في
----
((23, 23), (32, 32)) 0.447362499598187
to
في
----
((23, 23), (20, 20)) 0.3907348358576708
to
من
----
((23, 23), (4, 4)) 0.3719612262313735
to
إلى
----


In [ ]:
el_child_dict[((21, 23), (20, 21))]

(((21, 21), (21, 21)), ((23, 23), (20, 20)))

In [ ]:
for a,b in el_wt_dict.items():
  if a[0][1]>a[0][0]+3: continue
  print(a,b)
  src_str=" ".join(src0[a[0][0]:a[0][1]+1])
  trg_str=" ".join(trg0[a[1][0]:a[1][1]+1])
  print(src_str, trg_str)
  print("----")
  #print(trg0[a[1][0]:a[1][1]+1])

Streaming output truncated to the last 5000 lines.
not be used نظام لا يجوز استخدام ـها كـ ذريعة لقمع حريات أساسية وـ
----
((6, 9), (4, 14)) 0.5698722927490186
not be used as نظام لا يجوز استخدام ـها كـ ذريعة لقمع حريات أساسية وـ
----
((6, 7), (9, 14)) 0.19729898966059234
not be كـ ذريعة لقمع حريات أساسية وـ
----
((6, 7), (14, 15)) 0.18138297549981997
not be وـ حقوق
----
((6, 7), (7, 14)) 0.17561852475700077
not be استخدام ـها كـ ذريعة لقمع حريات أساسية وـ
----
((6, 7), (13, 14)) 0.17270720657494257
not be أساسية وـ
----
((6, 8), (7, 14)) 0.12807995997190613
not be used استخدام ـها كـ ذريعة لقمع حريات أساسية وـ
----
((6, 8), (10, 14)) 0.0807053147495788
not be used ذريعة لقمع حريات أساسية وـ
----
((6, 8), (9, 14)) 0.07230965281988545
not be used كـ ذريعة لقمع حريات أساسية وـ
----
((6, 8), (13, 14)) 0.06553552029024011
not be used أساسية وـ
----
((6, 9), (10, 14)) 0.16873566921923983
not be used as ذريعة لقمع حريات أساسية وـ
----
((6, 9), (7, 14)) 0.17509057404612044
not be used as استخ

In [ ]:
test_ch=((17, 18), (3, 16))
children0=el_child_dict.get(test_ch,[])
#children0=get_rec_el_children(test_ch,el_child_dict,[])
for ch0 in children0:
  print(ch0)
# print(span_top_elms_dict[(57,57)])
# print(start_span_dict[56])

((17, 17), (3, 3))
((17, 18), (8, 16))


In [ ]:
cur_items=list(span_child_dict.items())
for a in cur_items[:100]:
  print(a)

((1, 1), [])
((47, 47), [])
((47, 48), [(47, 48), (47, 47)])
((45, 45), [])
((45, 46), [(45, 45), (46, 46)])
((51, 51), [])
((51, 52), [(51, 52), (51, 51)])
((57, 57), [])
((57, 58), [(57, 57), (58, 58)])
((43, 43), [(43, 43), (43, 43)])
((49, 49), [(49, 49), (49, 49)])
((43, 44), [(43, 44), (43, 43)])
((49, 50), [(49, 50), (49, 49)])
((42, 42), [])
((42, 43), [(42, 43), (42, 42)])
((54, 54), [])
((54, 55), [(54, 54), (55, 55)])
((55, 55), [])
((55, 56), [(55, 55), (56, 56)])
((41, 41), [])
((41, 42), [(41, 41), (42, 42)])
((46, 46), [])
((46, 47), [(46, 46), (47, 47)])
((53, 53), [])
((53, 54), [(53, 53), (54, 54)])
((58, 58), [])
((0, 0), [])
((0, 1), [(0, 0), (1, 1)])
((48, 48), [])
((48, 49), [])
((21, 21), [(21, 21), (21, 21)])
((21, 22), [(21, 22), (21, 21)])
((31, 31), [])
((31, 32), [])
((27, 27), [])
((27, 28), [])
((2, 2), [])
((15, 15), [(15, 15), (15, 15)])
((19, 19), [(19, 19), (19, 19)])
((52, 52), [])
((52, 53), [])
((15, 16), [(15, 15), (16, 16)])
((19, 20), [(19, 19), 

In [ ]:
fopen=open("test-out-align.txt","w")
for a in all_phrase_pairs:
  line=" ".join([str(v) for v in a])+"\n"
  fopen.write(line)
fopen.close()
  #print(a)

In [ ]:
from collections import Counter
just_pairs=[]
for a in all_phrase_pairs:
  pair,sent_i=a
  src0,trg0=pair
  just_pairs.append(pair)
just_pairs.sort()
grouped=[(key,[v[1] for v in list(group)]) for key,group in groupby(just_pairs,lambda x:x[0])]
grouped.sort(key=lambda x:len(x[1]))
for k0,grp0 in grouped:
  print(k0)
  #print(grp0)
  print(Counter(grp0))
  print("----")

Streaming output truncated to the last 5000 lines.
Counter({'لـ قياس قدرة': 1})
----
mechanism
Counter({'آلية': 1})
----
mechanisms were
Counter({'كانت آليات': 1})
----
meeting
Counter({'اجتماع': 1})
----
meetings
Counter({'اجتماعات': 1})
----
member delegation concerned
Counter({'أعضاء وفد معني': 1})
----
member states at
Counter({'دول أعضاء في': 1})
----
member states at current meeting
Counter({'دول أعضاء في اجتماع حالي': 1})
----
member states but
Counter({'دول أعضاء ولكن': 1})
----
member states but was nevertheless
Counter({'دول أعضاء ولكن ـها مع ذلك': 1})
----
member states but was nevertheless essential to ensuring smooth operation organization </s>
Counter({'دول أعضاء ولكن ـها مع ذلك ضرورية لـ ضمان سلاسة عمل منظمة </s>': 1})
----
member states capacity
Counter({'قدرة دول أعضاء': 1})
----
member states capacity to
Counter({'إلى قدرة دول أعضاء': 1})
----
member states capacity to pay and on
Counter({'إلى قدرة دول أعضاء على دفع وـ': 1})
----
member states capacity to pay and on m

In [ ]:
def get_dist_factor(el1,el2): #[(5,6),(9,9)] - [(7,7),(12,13)] #get inner distance between two elements, 1 if adjacent
  el1_xrange,el1_yrange=el1
  el2_xrange,el2_yrange=el2
  combined_xs=[]
  combined_xs.extend(el1_xrange)
  combined_xs.extend(el2_xrange)
  combined_ys=[]
  combined_ys.extend(el1_yrange)
  combined_ys.extend(el2_yrange)
  combined_xs.sort()
  combined_ys.sort()
  inner_xdist=combined_xs[2]-combined_xs[1]
  inner_ydist=combined_ys[2]-combined_ys[1]
  return (inner_xdist+inner_ydist)/2
test_el1=[(5,6),(9,9)]
test_el2=[(6,6),(10,11)]
print(get_dist_factor(test_el1,test_el2))

0.5


In [ ]:
def overlaps(span1,span2):
  span1_i0,span1_i1=span1
  span2_i0,span2_i1=span2
  if span1_i0>=span2_i0 and span1_i0<=span2_i1: return True
  if span1_i1>=span2_i0 and span1_i1<=span2_i1: return True
  if span2_i0>=span1_i0 and span2_i0<=span1_i1: return True
  if span2_i1>=span1_i0 and span2_i1<=span1_i1: return True
  return False
sp1=(2,3)
sp2=(4,4)
print(overlaps(sp1,sp2))

False


In [ ]:
def get_inner_dist(span1,span2):
  combined_xs=[]
  combined_xs.extend(span1)
  combined_xs.extend(span2)  
  combined_xs.sort()
  return combined_xs[2]-combined_xs[1]
sp1=(22,30)
sp2=(14,16)
print(get_inner_dist(sp1,sp2))

6


In [ ]:
for ix in range(1,10):
  print(ix,math.log(ix))

1 0.0
2 0.6931471805599453
3 1.0986122886681098
4 1.3862943611198906
5 1.6094379124341003
6 1.791759469228055
7 1.9459101490553132
8 2.0794415416798357
9 2.1972245773362196


In [ ]:
import pandas as pd
xls_path='https://docs.google.com/spreadsheets/d/e/2PACX-1vSqipSUAaMWJrizvZtg1WIpds2pIiXtcZC6wjOLeVYie9p5O0lqiPnVdd8ambGKH8B0qJ4kmlVeEs6f/pub?output=xlsx'
cur_xls = pd.ExcelFile(xls_path)
cur_sheet_name="2"
cur_sheet_obj=pd.read_excel(cur_xls, cur_sheet_name,keep_default_na=False)
sent_pair_list=[]
for i,row in enumerate(cur_sheet_obj.iterrows()):
  row_dict=row[1].to_dict()
  sent_pair_list.append((row_dict["en"],row_dict["ar"]))

for a in sent_pair_list:
  print(a)
# print(cur_sheet_obj.keys())

('A/C.5/76/SR.2', 'A/C.5/76/SR.2')
('A/C.5/76/SR.2', 'A/C.5/76/SR.2')
('United Nations', 'الأمم المتحدة')
('A/C.5/76/SR.2', 'A/C.5/76/SR.2')
('General Assembly', 'الجمعية العامة')
('Seventy-sixth session', 'الدورة السادسة والسبعون')
('Official Records', 'وثيقة رسمية')
('Distr.: General', 'Distr.: General')
('24 January 2022', '24 January 2022')
('Original: English', 'Arabic Original: English')
('21-14241', '21-14241')
('This record is subject to correction.', 'هذا المحضر قابل للتصويب.')
('Corrections should be sent as soon as possible, under the signature of a member of the delegation concerned, to the Chief of the Documents Management Section (dms@un.org), and incorporated in a copy of the record.', 'وينبغي إدراج التصويبات في نسخة من المحضر مذيلة بتوقيع أحد أعضاء الوفد المعني وإرسالها في أقرب وقت ممكن إلى: Chief of the Documents Management Section (dms@un.org).')
('Corrected records will be reissued electronically on the Official Document System of the United Nations (http://documents

In [ ]:
def idx_match(idx1,idx2,min_common=5): #This function is to give a ratio of matching between the inverted indexes for two words
    if len(idx1)==0 or len(idx2)==0: return 0
    intersection=len(list(set(idx1).intersection(set(idx2))))
    union=len(idx1)+len(idx2)-intersection
    if union==0: return 0
    if intersection<min_common: return 0    
    return float(intersection)/union

test_idx1=[1,2,4,5,6,9]
test_idx2=[5,6,9,12,25]
idx_match(test_idx1,test_idx2,min_common=2)

0.375

In [ ]:
def filter_tokens(token_list0, lang="en",stop_words=[], ignore_punc=False,ignore_ar_pre_suf=False,remove_al=True,index_words=True,lower=True,stemming=False):
  tokens_original=list(token_list0)
  tokens_copy_enum=[(i,v) for i,v in enumerate(tokens_original)]   
  #if lang=="ar": tokens=tok_ar(tokens,count_dict) #tok_ar(tokens)
  if ignore_punc: tokens_copy_enum=[(i,v) for i,v in tokens_copy_enum if not is_punct(v)]
  if ignore_ar_pre_suf: tokens_copy_enum=[(i,v) for i,v in tokens_copy_enum if not v.startswith("ـ") and not v.endswith("ـ")]
  if stop_words: tokens_copy_enum=[(i,v) for i,v in tokens_copy_enum if not v.lower() in stop_words]
  if remove_al: tokens_copy_enum=[(i,v.replace("ال_","")) for i,v in tokens_copy_enum if not v.lower() in stop_words]
  tokens_out=[v[1] for v in tokens_copy_enum]
  if lower: tokens_out=[v.lower() for v in tokens_out]
  cur_mapping=[v[0] for v in tokens_copy_enum]
  return tokens_out,cur_mapping

sent='As only one country had benefited from that ceiling historically, it should be reviewed in accordance with General Assembly resolution 55/5 C.'
tokens0=tok(sent)
a,b=filter_tokens(tokens0, ignore_punc=True)
print(a)
print(b)

['as', 'only', 'one', 'country', 'had', 'benefited', 'from', 'that', 'ceiling', 'historically', 'it', 'should', 'be', 'reviewed', 'in', 'accordance', 'with', 'general', 'assembly', 'resolution', '55', '5', 'c']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24]


In [ ]:
#DUMP


  #     #print(first_items)
  #     #print(first_items[:3])
  #     #next_items=[]
  #     # next_items=list(first_items)
  #     # for tmp_i in range(i0+1,i1+1):
  #     #   next_items+=el_start_dict.get(tmp_i,[]) 
  #     #tmp_final_list=list(first_items)
  #     tmp_final_list=[] #the list of combined items with the first and next items
  #     #first_items.sort(key=lambda x:-x[-1])
  #     #next_items.sort(key=lambda x:-x[-1])
  #     #print(i0,i1)
  #     # print("first_items",len(first_items),"next_items",len(next_items))
  #     # print("first_items",first_items[:5],"next_items",next_items[:5])
      
  #     # for ni in next_items[-50:]:
  #     #   ni_element,ni_cur_ratio=ni
  #     #   print("NI",ni)
  #     # print("========")
  #     #for fi in first_items:
  #     cur_first_items=list(first_items.items())
  #     for fi in cur_first_items:
  #       fi_element,fi_cur_ratio=fi
  #       fi_src_range,fi_trg_range=fi_element
  #       fi_src_i0,fi_src_i1=fi_src_range
  #       fi_trg_i0,fi_trg_i1=fi_trg_range
  #       valid_next_items_counter=0
  #       next_items=list(cur_first_items)
  #       for i_plus in range(fi_src_i1+1,fi_src_i1+5):
  #         #next_items+=el_start_dict.get(i_plus,[]) 
  #         test_next=el_start_dict.get(i_plus,{})
  #         for nx in test_next.items(): next_items.append(nx)

  #         #print("test_next",test_next)
        

  #       # print(fi, len(next_items))
  #       # print(next_items[:3])
  #       # print("-------")
  #       # continue

  #       for ni in next_items:
  #         ni_element,ni_cur_ratio=ni
  #         if fi_element==ni_element: continue
  #         ni_src_range,ni_trg_range=ni_element
  #         ni_src_i0,ni_src_i1=ni_src_range
  #         ni_trg_i0,ni_trg_i1=ni_trg_range   
  #         if ni_src_i0>fi_src_i1+10: continue

  #         trg_spans_overlap=overlaps(fi_trg_range,ni_trg_range)
  #         if trg_spans_overlap: continue #skip if target spans overlap

  #         trg_span_inner_dist=get_inner_dist(fi_trg_range,ni_trg_range)
  #         if trg_span_inner_dist>10: continue #skip if there is much distance between target spans

  #         src_span_inner_dist=get_inner_dist(fi_src_range,ni_src_range)
          
          
          
  #         if fi_src_i0==fi_src_i1==ni_src_i0 and ni_src_i0!=ni_src_i1: continue #vertical element with non-vertical element
  #         #if ni_src_i0<=fi_src_i1: continue #skip if src spans overlap
          
          
  #         #if src_span_inner_dist>10: continue #skip if there is much distance between source spans
  #         if fi_src_range==ni_src_range: 
  #           if fi_src_i0!=fi_src_i1: continue
  #           if ni_trg_i0<fi_trg_i0: continue
  #           #print(">>>>> Vertical element",fi_element,ni_element)

  #         el_pair=(fi_element,ni_element)
  #         if used_el_dict.get(el_pair,False)==True: continue
  #         used_el_dict[el_pair]=True

  #         valid_next_items_counter+=1

  #         new_src_range=get_min_max(fi_src_range,ni_src_range)
  #         new_trg_range=get_min_max(fi_trg_range,ni_trg_range)
  #         combined_ratio=fi_cur_ratio+ni_cur_ratio
  #         cur_area_factor=get_area_factor(new_src_range,new_trg_range)

  #         if cur_area_factor<1: continue
  #         #adj_ratio=combined_ratio/cur_area_factor
  #         #adj_ratio=combined_ratio-trg_span_inner_dist*0.05-src_span_inner_dist*0.05-cur_area_factor*0.005
  #         adj_ratio=combined_ratio
  #         adj_wt=combined_ratio
  #         cur_el=(new_src_range,new_trg_range)
  #         found_span_wt=span_wt_dict.get(new_src_range,0)
  #         if adj_wt>found_span_wt:


  #           el_start_dict[i0][cur_el]=adj_wt
  #           span_wt_dict[new_src_range]=adj_wt
  #           span_el_dict[new_src_range]=cur_el
  #           # print("fi_element,fi_cur_ratio",fi)
  #           # print("ni_element,ni_cur_ratio",ni)
  #           # print("cur_el,adj_wt",cur_el,adj_wt)
  #           # print("el_start_dict[i0][cur_el]",el_start_dict[i0].get(cur_el,0))
  #           # print("UPDATED el_start_dict[i0][cur_el]",el_start_dict[i0].get(cur_el,0))
  #           # print("------------------")

  #           if new_src_range!=fi_src_range and new_src_range!=ni_src_range:  
  #             span_child_dict[new_src_range]=[fi_src_range,ni_src_range]
  #         found_ratio=tmp_ratio_dict.get(cur_el,0)
  #         if adj_ratio>found_ratio:
  #           tmp_final_list.append((cur_el,adj_ratio))
      
  #     # print("first_items",len(first_items),"next_items",len(next_items),"valid",valid_next_items_counter)
  #     # print("used_el_dict", len(used_el_dict.keys()))
  #     # if valid_next_items_counter==0:
  #     #   for fi in first_items:
  #     #     print(fi)
      
  #     tmp_final_list.sort(key=lambda x:-x[-1])
  #     used_els=[]
  #     final_items=[]
  #     for tp in tmp_final_list:
  #       el0,ratio0=tp
  #       if el0 in used_els: continue
  #       used_els.append(el0)
  #       final_items.append(tp)
  #       src_range0,trg_range0=tp[0]        
  #       cur_src_str=" ".join([src0[i_] for i_ in range(src_range0[0],src_range0[1]+1)])
  #       cur_trg_str=" ".join([trg0[i_] for i_ in range(trg_range0[0],trg_range0[1]+1)])

  #     final_items.sort(key=lambda x:-x[-1])
  #     #el_start_dict[i0]=first_items+final_items[:5] ##### CHECK

  # # list_spans_wts=[]
  # # for a,b in span_wt_dict.items():
  # #   list_spans_wts.append((a,b))
  
  # # list_spans_wts.sort(key=lambda x:-x[-1])

  # # #get_rec_children(span0,span_child_dict0,span_el_dict0,el_list0=[])
  # # print(list_spans_wts[0])
  # tmp_full_span_wt=span_wt_dict.get(full_src_span,0)
  #all_elements=get_rec_children(full_src_span,span_child_dict,span_el_dict,el_list0=[])

#Translation/QA from reference bitexts

In [ ]:
#Preparing references
import pandas as pd
import re
def get_bitext(xls_path0,sheet_name0="",src_col="en",trg_col="ar",filter_non_alpha=False, filter_numbers=False):
  #xls_path='https://docs.google.com/spreadsheets/d/e/2PACX-1vSVy7hY4fV0LHqTCypOjZAM9OrBevDDnpIcEKEAPKDsYRoxAz9TUBcDIgy8PvHkw5IDAZ6eRmKdZuRw/pub?output=xlsx'
  cur_xls = pd.ExcelFile(xls_path0)
  bitext_list=[]
  if sheet_name0!="": cur_sheet_list=[sheet_name0]
  else: cur_sheet_list=cur_xls.sheet_names
  for cur_sheet_name in cur_sheet_list:
    print(cur_sheet_name)
    cur_sheet_obj=pd.read_excel(cur_xls, cur_sheet_name,keep_default_na=False)
    cur_sheet_obj[src_col]=cur_sheet_obj[src_col].apply(str)
    cur_sheet_obj[trg_col]=cur_sheet_obj[trg_col].apply(str)
    for i,row in enumerate(cur_sheet_obj.iterrows()):
      row_dict=row[1].to_dict()
      en0,ar0=row_dict["en"],row_dict["ar"]
      test_alpha=re.sub("\W+","",en0)
      if filter_non_alpha and test_alpha=="": continue
      if filter_numbers and test_alpha.isdigit(): continue
      if trg_col=="ar": ar0=clean_ar(ar0)
      bitext_list.append((en0,ar0))
  return bitext_list


def get_key(txt):
  txt=re.sub("\W+","_",txt)
  txt=txt.strip("_")
  return txt.lower()
#Ref bitexts
xls_path='https://docs.google.com/spreadsheets/d/e/2PACX-1vSVy7hY4fV0LHqTCypOjZAM9OrBevDDnpIcEKEAPKDsYRoxAz9TUBcDIgy8PvHkw5IDAZ6eRmKdZuRw/pub?output=xlsx'
cur_xls = pd.ExcelFile(xls_path)
translation_dict={}
for cur_sheet_name in cur_xls.sheet_names:
  print(cur_sheet_name)
  cur_sheet_obj=pd.read_excel(cur_xls, cur_sheet_name,keep_default_na=False)
  cur_sheet_obj['en']=cur_sheet_obj['en'].apply(str)
  cur_sheet_obj['ar']=cur_sheet_obj['ar'].apply(str)

  sent_pair_list=[]
  key_str_pairs=[]
  for i,row in enumerate(cur_sheet_obj.iterrows()):
    row_dict=row[1].to_dict()
    en0,ar0=row_dict["en"],row_dict["ar"]
    en_key=get_key(en0)
    test_alpha=re.sub("\W+","",en0)
    if test_alpha=="": continue
    if test_alpha.isdigit(): continue

    key_str_pairs.append((en_key,en_key))
    #if en0.replace(".","").replace("-","").replace("/","").isdigit(): continue
    ar0=clean_ar(ar0)
    tmp_translation_dict=translation_dict.get(en_key,{})
    tmp_translation_dict[ar0]=tmp_translation_dict.get(ar0,0)+1
    translation_dict[en_key]=tmp_translation_dict
    sent_pair_list.append((en0,ar0))
    src_sents.append(en0)
    trg_sents.append(ar0)
  #for a in sent_pair_list[:20]: print(a)
test_items=list(translation_dict.items())
test_items.sort(key=lambda x:-sum(x[1].values()))
for a,b in test_items[:200]:
  b_items=list(b.items())
  b_items.sort(key=lambda x:-x[1])
  for a1,b1 in b_items:
    print(a,a1,b1)
  print("-----")

A75760


NameError: ignored

In [ ]:
from itertools import groupby
all_ref_keys=list(translation_dict.keys())
first_word_list=[(v.split("_")[0],v) for v in all_ref_keys]
first_word_list.sort()
first_word_list_grouped=[(key,[v[1] for v in list(group)]) for key,group in groupby(first_word_list,lambda x:x[0])]
first_word_dict=dict(iter(first_word_list_grouped))
first_word_dict_items=list(first_word_dict.items())
print(len(first_word_dict_items))
for a,b in first_word_dict_items[1000:1020]:
  print(a,b)
#print(all_ref_keys[:20])

1377
ptsd ['ptsd', 'ptsd_claims_are_assessed_and_processed_within_90_days_of_receipt_of_required_documentation_percentage', 'ptsd_claims_reviewed_and_processed_within_90_days_after_receipt_of_required_documentation_percentage']
public ['public_awareness_raising_activity_on_the_protection_monitoring_and_reporting_mechanism_on_grave_violations_against_children_in_situations_of_armed_conflicts_for_2_594_participants_including_552_women', 'public_events', 'public_events_were_conducted_pending_the_publication_of_the_report_on_human_rights_trends', 'public_health', 'public_information_and_publication_services', 'public_information_and_publication_services_including_subscriptions', 'public_information_and_publication_services_translation', 'public_information_campaigns_on_the_electoral_cycle_process_based_on_the_scope_of_the_mission_s_mandate_aimed_at_informing_malian_citizens_and_voters_about_the_electoral_procedures_to_prevent_conflicts_triggered_by_the_electoral_process_through_60_radio_pr

In [ ]:
from code_utils.general import * 
import code_utils.general as utils

job_xls_path='https://docs.google.com/spreadsheets/d/e/2PACX-1vSqipSUAaMWJrizvZtg1WIpds2pIiXtcZC6wjOLeVYie9p5O0lqiPnVdd8ambGKH8B0qJ4kmlVeEs6f/pub?output=xlsx'
cur_sheet_name="current"
cur_job_bitext=get_bitext(job_xls_path,sheet_name0=cur_sheet_name,src_col="en",trg_col="ar",filter_non_alpha=True, filter_numbers=True)
cur_terminology_bitext=get_bitext(job_xls_path,sheet_name0="terminology",src_col="en",trg_col="ar",filter_non_alpha=True, filter_numbers=True)
src_only=[v[0] for v in cur_job_bitext]
xl8_list=[]
for en0 in src_only:
  match_type="" #exact - terminology - template - linear - MT ... ??
  en_key0=get_key(en0) #first check the exact match
  eq=translation_dict.get(en_key0,{})
  eq_items=list(eq.items())
  eq_items.sort(key=lambda x:-x[1])  
  selected_eq=""
  if eq_items: 
    selected_eq=eq_items[0][0]
    match_type="exact"
  if match_type!="": continue
  print(en0)
  
  
  en_key0_split=en_key0.split("_")
  words=list(set(en_key0_split))
  valid_keys_tokens=[]
  for w in words:
    corr_keys=first_word_dict.get(w,[])
    for k0 in corr_keys:
      k0_split=k0.split("_")
      if list_in_list(k0_split,en_key0_split): valid_keys_tokens.append(k0)
  valid_keys_tokens.sort(key=lambda x:-len(x[0]))
  print(valid_keys_tokens)
  tmp_linear_str=" ".join(en_key0_split)
  for vk in valid_keys_tokens:
    key_str=vk.replace("_"," ")
    corr=translation_dict.get(vk,{})
    corr_items=sorted(list(corr.items()),key=lambda x:-x[1])
    top_corr_str=corr_items[0][0]
    tmp_linear_str=re.sub(r'\b%s\b'%key_str,top_corr_str,tmp_linear_str)
    #print(vk, corr_items[0])
  print(tmp_linear_str)

  
  xl8_list.append((en0,selected_eq,match_type))




test_out_fopen=open("xl8_out.txt","w")
for item in xl8_list:
  test_out_fopen.write("\t".join(item)+"\n")
test_out_fopen.close()  


current
terminology
A/76/xxx
['a']
(أ) 76 xxx
A/76/XXX
['a']
(أ) 76 xxx
MINUSCA will seek to strengthen its interregional cooperation and economies of scale with missions in the region, including the United Nations Regional Office for Central Africa (UNOCA), to enhance collaboration, information-sharing and analysis, and facilitation, as appropriate.
['united_nations', 'office', 'minusca']
بعثة الأمم المتحدة المتكاملة المتعددة الأبعاد لتحقيق الاستقرار في جمهورية أفريقيا الوسطى will seek to strengthen its interregional cooperation and economies of scale with missions in the region including the الأمم المتحدة regional المكتب for central africa unoca to enhance collaboration information sharing and analysis and facilitation as appropriate
MINUSCA will seek to leverage existing assets of other missions following their downsizing and withdrawal to meet its operational requirements, if any.
['other', 'minusca']
بعثة الأمم المتحدة المتكاملة المتعددة الأبعاد لتحقيق الاستقرار في جمهورية أفريقيا

#OLD

In [ ]:

#Job Bitexts
xls_path='https://docs.google.com/spreadsheets/d/e/2PACX-1vSqipSUAaMWJrizvZtg1WIpds2pIiXtcZC6wjOLeVYie9p5O0lqiPnVdd8ambGKH8B0qJ4kmlVeEs6f/pub?output=xlsx'
cur_xls = pd.ExcelFile(xls_path)
cur_sheet_name="current"
cur_job_bitext=get_bitext(xls_path,sheet_name0=cur_sheet_name,src_col="en",trg_col="ar",filter_non_alpha=True, filter_numbers=True)

cur_sheet_obj=pd.read_excel(cur_xls, cur_sheet_name,keep_default_na=False)
cur_sheet_obj['en']=cur_sheet_obj['en'].apply(str)
cur_sheet_obj['ar']=cur_sheet_obj['ar'].apply(str)

job_pair_list=[]
for i,row in enumerate(cur_sheet_obj.iterrows()):
  row_dict=row[1].to_dict()
  en0,ar0=row_dict["en"],row_dict["ar"]
  en_key=get_key(en0)

  
  if en_key.replace("_","")=="": continue
  if en_key.replace("_","").isdigit(): continue

  # print(en_key)
  # print(en0)
  # print("------")

  ar0=clean_ar(ar0)
  job_pair_list.append((en_key,en0,ar0))
translated_list=[]
used_en=[]
for en_key0,en0,ar0 in job_pair_list:
  #continue
  
  # if en_key0 in used_en: continue
  # used_en.append(en_key0)
  #print(en0)
  eq=translation_dict.get(en_key0,{})
  eq_items=list(eq.items())
  eq_items.sort(key=lambda x:-x[1])  
  selected_eq=""
  if eq_items: selected_eq=eq_items[0][0]
  translated_list.append((en0,selected_eq))
  # for eq_ar0 in eq_items:
  #   print(eq_ar0)
  # print("----")

test_out_fopen=open("xl8_out.txt","w")
for item in translated_list:
  test_out_fopen.write("\t".join(item)+"\n")
test_out_fopen.close()  


#Classification/embedding with gensim

In [ ]:
#Create a model
import time, os, re
#from code_utils.general import *
from gensim.models import Word2Vec

def tok(txt):
  return re.findall("\w+",txt)


en_corpus_fname="UNv1.0.ar-en.en"

n_sents_max=500000
en_fopen=open(en_corpus_fname)

src_sents=[]
for i,en_line in enumerate(en_fopen):
  if i>=n_sents_max: break
  if i%50000==0: print(i)
  en_line=en_line.strip("\n\r\t ")
  toks=tok(en_line.lower())
  if len(toks)<3: continue
  src_sents.append(toks)

#ar_fopen.close()
en_fopen.close()


# def get_sentences(fpath):
#   fopen=open(fpath)
#   tmp_sentences=[]
#   sent_counter=0
#   completed=False
#   for i,site_content in enumerate(fopen):
#     if completed: break
#     content_split=site_content.split("<br>")
#     #print(len(content_split), content_split[0])
#     if len(content_split)<4: continue
#     for line in content_split:
#       line=line.strip("\n\r\t ")
#       toks=tok(line.lower())
#       if len(toks)<3: continue
#       sent_counter+=1
#       if sent_counter%10000==0: print(sent_counter)
#       if sent_counter>2000000: 
#         completed=True
#         break
#       tmp_sentences.append(toks)
#       #yield toks
#   fopen.close()
#   return tmp_sentences

model_fname="un_en.wv"

# t0=time.time()
# data_fpath="crawl/oct21_au/cached_all.txt"
t0=time.time()
#sentences=get_sentences(data_fpath)
print("collected %s sentences"%len(src_sents))
print("Now creating the WV model")
un_model = Word2Vec(src_sents, size=100, min_count=5) #small model
t1=time.time()
print("collected sentences in %s seconds"%(t1-t0))
#fopen.close()

un_model.save(model_fname)
print("saved model as: ",model_fname)

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
collected 474505 sentences
Now creating the WV model
collected sentences in 90.76132202148438 seconds
saved model as:  un_en.wv


In [ ]:
import gensim
from collections import Counter
from scipy import spatial

def get_words_vector(words,wv_model,excluded_words=[]):
  wd_counter=Counter(words)
  flag=False
  total_count0=0
  wd_vector_dict={}
  for wd0,wd0_n in wd_counter.items():
    if wd0 in excluded_words: continue
    try: cur_vec0=wv_model[wd0]
    except: continue
    wd_vector_dict[wd0]=cur_vec0
    weighted_vec0=cur_vec0*wd0_n
    if flag==False: total_vec0=weighted_vec0
    else: total_vec0+=weighted_vec0
    flag=True
    total_count0+=wd0_n
  if total_count0==0: return [],{}
  avg_vec=total_vec0/total_count0
  return avg_vec,wd_vector_dict

#cosine similarity between word vectors
def cos_sim(vector1,vector2):
  result = 1 - spatial.distance.cosine(vector1, vector2)
  return result

un_model = gensim.models.Word2Vec.load("un_en.wv")
#un_model["about"]
un_model.most_similar("agriculture")

cur_sent='To reduce transnational threats and enhance stabilization efforts, the Mission will continue to work closely with regional actors for the full operationalization of joint mixed commissions and fully functional cross-border mechanisms, in close cooperation with UNOCA.'
cur_tokens=tok(cur_sent)
sent_vec,_=get_words_vector(cur_tokens,un_model)
term1='United Nations Regional Office for Central Africa'
term2='United Nations Operations Centre in Afghanistan'

terms=['United Nations Regional Office for Central Africa','United Nations Operations Centre in Afghanistan',"Peace and Security","Human Rights","Environment","Spain","General Assembly","International Law"]
for tr in terms:
  term_tokens=tok(tr.lower())
  term_vec,_=get_words_vector(term_tokens,un_model)
  sim0=cos_sim(sent_vec,term_vec)
  print(tr,sim0)

# term1='Security'
# term2='Environment'
# term1_tokens=tok(term1.lower())
# term2_tokens=tok(term2.lower())
# term1_vec,_=get_words_vector(term1_tokens,un_model)
# term2_vec,_=get_words_vector(term2_tokens,un_model)
# print(cos_sim(sent_vec,term1_vec))
# print(cos_sim(sent_vec,term2_vec))



# term1_tokens=tok(term1.lower())
# term2_tokens=tok(term2.lower())
# term1_vec,_=get_words_vector(term1_tokens,un_model)
# term2_vec,_=get_words_vector(term2_tokens,un_model)
# print(cos_sim(sent_vec,term1_vec))
# print(cos_sim(sent_vec,term2_vec))


United Nations Regional Office for Central Africa 0.6001240015029907
United Nations Operations Centre in Afghanistan 0.538229763507843
Peace and Security 0.45091143250465393
Human Rights 0.27792084217071533
Environment 0.26981398463249207
Spain -0.17120014131069183
General Assembly -0.005136891268193722
International Law 0.3549458980560303


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


#Parsing and Segmentation Experiments

In [ ]:
from code_utils.parsing_lib import *
from code_utils.dep_lib import *


In [ ]:
sent1="""
At the same time, in line with the Secretary-General call for operating as ‘One UN’ and as part of DOS/OSCM, the Centre is responding to demands to achieve system-wide efficiencies and to deliver solutions to a broader client base that includes the wider United Nations system.
"""
conll0=get_conll(sent1.strip())
for c_line in conll0:
  print("\t".join([str(v) for v in c_line]))

1	At	at	IN	ADP	_	0	ROOT	_	_
2	the	the	DT	DET	_	4	det	_	_
3	same	same	JJ	ADJ	_	4	amod	_	_
4	time	time	NN	NOUN	_	1	pobj	_	_
5	,	,	,	PUNCT	_	1	punct	_	_
6	in	in	IN	ADP	_	5	intj	_	_
7	line	line	NN	NOUN	_	6	pobj	_	_
8	with	with	IN	ADP	_	7	prep	_	_
9	the	the	DT	DET	_	13	det	_	_
10	Secretary	Secretary	NNP	PROPN	_	12	compound	_	_
11	-	-	HYPH	PUNCT	_	12	punct	_	_
12	General	General	NNP	PROPN	_	13	compound	_	_
13	call	call	NN	NOUN	_	8	pobj	_	_
14	for	for	IN	ADP	_	13	prep	_	_
15	operating	operate	VBG	VERB	_	14	pcomp	_	_
16	as	as	IN	SCONJ	_	15	prep	_	_
17	‘	'	``	PUNCT	_	19	punct	_	_
18	One	one	CD	NUM	_	19	nummod	_	_
19	UN	UN	NNP	PROPN	_	16	pobj	_	_
20	’	'	''	PUNCT	_	19	punct	_	_
21	and	and	CC	CCONJ	_	32	cc	_	_
22	as	as	IN	SCONJ	_	32	prep	_	_
23	part	part	NN	NOUN	_	22	pobj	_	_
24	of	of	IN	ADP	_	23	prep	_	_
25	DOS	DOS	NNP	PROPN	_	27	nmod	_	_
26	/	/	SYM	SYM	_	27	punct	_	_
27	OSCM	OSCM	NNP	PROPN	_	24	pobj	_	_
28	,	,	,	PUNCT	_	32	punct	_	_
29	the	the	DT	DET	_	30	det	_	_
30	Centre	Centre	NNP	PROPN	_	32	

In [ ]:
# for c_line in conll0:
#   print("\t".join([str(v) for v in c_line])+"\n")
link_count_dict={}
side_dict={}
tok_dict={}
left_emanating_dict={}
right_emanating_dict={}

token_list=[v[1] for v in conll0]
for c_i,c_line in enumerate(conll0):
  cur_token=c_line[1]
  cur_tag=c_line[4]
  cur_i=int(c_line[0])
  cur_parent_i=int(c_line[6]) 
  tok_dict[cur_i]=cur_token
  
  if cur_parent_i==0: cur_side="0"
  elif cur_parent_i>cur_i: cur_side="R"
  elif cur_parent_i<cur_i: cur_side="L"
  side_dict[cur_i]=cur_side
  if cur_side=="R": left_emanating_dict[cur_parent_i]=left_emanating_dict.get(cur_parent_i,0)+1
  if cur_side=="L": right_emanating_dict[cur_parent_i]=right_emanating_dict.get(cur_parent_i,0)+1
  #let's process links - count how many links between two consecutive words
  if cur_tag=='PUNCT' and c_i==len(conll0)-1: continue #ignore last token from counting links if it is punctuation 
  if cur_parent_i==0: continue 
  min_i,max_i=min(cur_i,cur_parent_i),max(cur_i,cur_parent_i)
  for tmp_i in range(min_i,max_i):
    link_count_dict[tmp_i]=link_count_dict.get(tmp_i,0)+1

boundary_i_list=[0]
for i0 in range(1,len(conll0)-1):
  i1=i0+1
  cur_link_count=link_count_dict.get(i0,0)
  #print(tok_dict[i0],tok_dict[i1], ">> n links:",cur_link_count)
  side1,side2=side_dict[i0],side_dict[i1]
  #print("sides:",side1,side2)
  #if (side1,side2)==("L","R") or cur_link_count==1: boundary_i_list.append(i0)
  if (side1,side2)!=("L","R"): continue
  if left_emanating_dict.get(i1,0)>0: continue
  if right_emanating_dict.get(i0,0)>0: continue
  boundary_i_list.append(i0)
  #if (side1,side2)==("L","R"): 
    
boundary_i_list.append(len(conll0))
for i0,i1 in zip(boundary_i_list,boundary_i_list[1:]):
  cur_slice=" ".join(token_list[i0:i1])
  print(i0,i1, cur_slice)
  #print(i0,i1)
  #print()
  #print(side_dict[i0],side_dict[i1])
  
  print("----")
# for c_i,c_line in conll0:
#   #print(a, side_dict[cur_i])
#   cur_i0=int(c_line[0])


0 20 At the same time , in line with the Secretary - General call for operating as ‘ One UN ’
----
20 27 and as part of DOS / OSCM
----
27 41 , the Centre is responding to demands to achieve system - wide efficiencies and
----
41 57 to deliver solutions to a broader client base that includes the wider United Nations system .
----


In [ ]:
from code_utils.parsing_lib import *
from code_utils.dep_lib import *

def conll2seg(conll0_input):
  link_count_dict={}
  side_dict={}
  tok_dict={}
  left_emanating_dict={}
  right_emanating_dict={}

  token_list=[v[1] for v in conll0_input]
  for c_i,c_line in enumerate(conll0_input):
    cur_token=c_line[1]
    cur_tag=c_line[4]
    cur_i=int(c_line[0])
    cur_parent_i=int(c_line[6]) 
    tok_dict[cur_i]=cur_token
    
    if cur_parent_i==0: cur_side="0"
    elif cur_parent_i>cur_i: cur_side="R"
    elif cur_parent_i<cur_i: cur_side="L"
    side_dict[cur_i]=cur_side
    if cur_side=="R": left_emanating_dict[cur_parent_i]=left_emanating_dict.get(cur_parent_i,0)+1
    if cur_side=="L": right_emanating_dict[cur_parent_i]=right_emanating_dict.get(cur_parent_i,0)+1
    #let's process links - count how many links between two consecutive words
    if cur_tag=='PUNCT' and c_i==len(conll0_input)-1: continue #ignore last token from counting links if it is punctuation 
    if cur_parent_i==0: continue 
    min_i,max_i=min(cur_i,cur_parent_i),max(cur_i,cur_parent_i)
    for tmp_i in range(min_i,max_i):
      link_count_dict[tmp_i]=link_count_dict.get(tmp_i,0)+1

  boundary_i_list=[0]
  for i0 in range(1,len(conll0_input)-1):
    i1=i0+1
    cur_link_count=link_count_dict.get(i0,0)
    #print(tok_dict[i0],tok_dict[i1], ">> n links:",cur_link_count)
    side1,side2=side_dict[i0],side_dict[i1]
    #print("sides:",side1,side2)
    #if (side1,side2)==("L","R") or cur_link_count==1: boundary_i_list.append(i0)
    if (side1,side2)!=("L","R"): continue
    if left_emanating_dict.get(i1,0)>0: continue
    if right_emanating_dict.get(i0,0)>0: continue
    boundary_i_list.append(i0)
    #if (side1,side2)==("L","R"): 
      
  boundary_i_list.append(len(conll0_input))
  all_slices=[]
  for i0,i1 in zip(boundary_i_list,boundary_i_list[1:]):
    cur_slice=" ".join(token_list[i0:i1])
    all_slices.append(cur_slice)
    #print(i0,i1, cur_slice)  
  return all_slices
sent1="""
This proposal is based on the vision of supply chain management that guides OSCM operations and the strengthening of core digital technology capacities to increase the Organization’s resilience and effectiveness and to contribute to the Secretary-General’s Data and New Technologies Strategies.
"""

conll0=get_conll(sent1.strip())
for c_line in conll0:
  print("\t".join([str(v) for v in c_line]))

segs=conll2seg(conll0)
for s0 in segs:
  print(s0) 

1	This	this	DT	DET	_	2	det	_	_
2	proposal	proposal	NN	NOUN	_	4	nsubjpass	_	_
3	is	be	VBZ	AUX	_	4	auxpass	_	_
4	based	base	VBN	VERB	_	0	ROOT	_	_
5	on	on	IN	ADP	_	4	prep	_	_
6	the	the	DT	DET	_	7	det	_	_
7	vision	vision	NN	NOUN	_	5	pobj	_	_
8	of	of	IN	ADP	_	7	prep	_	_
9	supply	supply	NN	NOUN	_	10	compound	_	_
10	chain	chain	NN	NOUN	_	11	compound	_	_
11	management	management	NN	NOUN	_	8	pobj	_	_
12	that	that	WDT	DET	_	13	nsubj	_	_
13	guides	guide	VBZ	VERB	_	11	relcl	_	_
14	OSCM	OSCM	NNP	PROPN	_	15	compound	_	_
15	operations	operation	NNS	NOUN	_	13	dobj	_	_
16	and	and	CC	CCONJ	_	7	cc	_	_
17	the	the	DT	DET	_	18	det	_	_
18	strengthening	strengthening	NN	NOUN	_	7	conj	_	_
19	of	of	IN	ADP	_	18	prep	_	_
20	core	core	JJ	ADJ	_	23	amod	_	_
21	digital	digital	JJ	ADJ	_	22	amod	_	_
22	technology	technology	NN	NOUN	_	23	compound	_	_
23	capacities	capacitie	VBZ	VERB	_	19	pobj	_	_
24	to	to	TO	PART	_	25	aux	_	_
25	increase	increase	VB	VERB	_	18	acl	_	_
26	the	the	DT	DET	_	27	det	_	_
27	Organization	Organi

In [ ]:
xls_path0='https://docs.google.com/spreadsheets/d/e/2PACX-1vSqipSUAaMWJrizvZtg1WIpds2pIiXtcZC6wjOLeVYie9p5O0lqiPnVdd8ambGKH8B0qJ4kmlVeEs6f/pub?output=xlsx'
cur_bitext=get_bitext(xls_path0,sheet_name0="current")
src_list=[v[0] for v in cur_bitext]
for s_i, sent0 in enumerate(src_list):
  print(s_i, "::",sent0)
  cur_conll0=get_conll(sent0)
  segs0=conll2seg(cur_conll0)
  for s_ in segs0:
    print(">",s_)
  print("---")


current
0 :: A/75/769xxx
> A/75/769xxx
---
1 :: A/76/xxx
> A/76 / xxx
---
2 :: United Nations
> United Nations
---
3 :: A/76/730
> A/76/730
---
4 :: General Assembly
> General Assembly
---
5 :: Distr.: General
> Distr .
> : General
---
6 :: 2022-03-01 00:00:00
> 2022 - 03 - 01 00:00:00
---
7 :: Original: English
> Original : English
---
8 :: f
> f
---
9 :: Seventy-sixth session
> Seventy - sixth session
---
10 :: Agenda item 150
> Agenda item 150
---
11 :: Administrative and budgetary aspects of the financing of the United Nations peacekeeping operations
> Administrative and budgetary aspects of the financing of the United Nations peacekeeping operations
---
12 :: Budget for the United Nations Logistics Base at Brindisi, Italy, for the period from 1 July 2022 to 30 June 2023
> Budget for the United Nations Logistics Base at Brindisi , Italy , for the period from 1 July 2022 to 30 June 2023
---
13 :: Report of the Secretary-General
> Report of the Secretary - General
---
14 :: Contents


#Testing New approach for hashing matching

In [ ]:
ar_corpus_fname="UNv1.0.ar-en-clean2.ar"
en_corpus_fname="UNv1.0.ar-en.en"

n_sents_max=20000
ar_fopen=open(ar_corpus_fname)
en_fopen=open(en_corpus_fname)

src_counter,trg_counter={},{}
src_sents,trg_sents=[],[]
for i,pair in enumerate(zip(en_fopen,ar_fopen)):
  if i>=n_sents_max: break
  if i%20000==0: print(i)
  en_line,ar_line=pair
  en_line,ar_line=en_line.strip("\n\r\t "),ar_line.strip("\n\r\t ")
  # src_sents.append(en_line)
  # trg_sents.append(ar_line)
  src_tokens=tok(en_line)
  trg_tokens=cur_ar_tok_fn(ar_line)
  src_sents.append(src_tokens)
  trg_sents.append(trg_tokens)
  for a in src_tokens:src_counter[a]=src_counter.get(a,0)+1
  for a in trg_tokens:trg_counter[a]=trg_counter.get(a,0)+1
  # print(en_tokens)
  # print(ar_tokens)
  # print("---")
print("len src tokens=",len(src_counter.keys()))
print("len trg tokens=",len(trg_counter.keys()))

0
len src tokens= 16599
len trg tokens= 21113


In [ ]:
import gensim
print("working on src")
en_model=gensim.models.Word2Vec(
        src_sents,
        size=100,
        window=10,
        min_count=2,
        workers=10,
        iter=10)
print("working on trg")
ar_model=gensim.models.Word2Vec(
        trg_sents,
        size=100,
        window=10,
        min_count=2,
        workers=10,
        iter=10)

working on src
working on trg


In [ ]:
src_keys=list(src_counter.keys())
print(src_keys[100:110])
trg_keys=list(trg_counter.keys())
print(trg_keys[100:110])

['Code', 'Crimes', 'against', 'Peace', 'Security', 'Mankind', 'which', 'was', 'devoted', 'to']
['ال_جرائم', 'ال_مخلة', 'سلم', 'ال_إنسانية', 'أمن', 'ـها', 'الذي', 'كرس', 'لـ', 'إمكانية']


In [ ]:
from scipy import spatial
# for a in trg_keys[:10]:
#   vec=ar_model[a]
  # src_sents.append(src_tokens)
  # trg_sents.append(trg_tokens)
for src0,trg0 in zip(src_sents[:20],trg_sents[:20]):
  tmp_src_vec_dict,tmp_trg_vec_dict={},{}
  try: 
    for a in src0: tmp_src_vec_dict[a]=en_model[a]
  except: pass
  try: 
    for a in trg0: tmp_trg_vec_dict[a]=ar_model[a]
  except: pass
  for a0,b0 in tmp_src_vec_dict.items():
    tmp=[]
    print(a0)
    for a1,b1 in tmp_trg_vec_dict.items():
      
      cos_val = abs( 1 - spatial.distance.cosine(b0, b1))
      tmp.append((a1,cos_val))
      #print(a0,a1,cos_val)
    tmp.sort(key=lambda x:-x[1])
    for x in tmp[:5]:
      print(x)
    print("----")



ORIGINAL
('/', 0.1210264042019844)
(':', 0.09776163846254349)
('ENGLISH', 0.09312044829130173)
('SPANISH', 0.07970868051052094)
('ORIGINAL', 0.07445640116930008)
----
:
('/', 0.1281573325395584)
(':', 0.10443489998579025)
('ENGLISH', 0.09897531569004059)
('SPANISH', 0.08429171144962311)
('ORIGINAL', 0.07810582965612411)
----
ENGLISH
('/', 0.12239494174718857)
(':', 0.1018538549542427)
('ENGLISH', 0.09902690351009369)
('SPANISH', 0.08270221203565598)
('ORIGINAL', 0.07603658735752106)
----
/
('/', 0.1405307948589325)
(':', 0.1113162711262703)
('ENGLISH', 0.10568653047084808)
('SPANISH', 0.09063097089529037)
('ORIGINAL', 0.08553128689527512)
----
SPANISH
('/', 0.10009922087192535)
(':', 0.07723880559206009)
('SPANISH', 0.07304131239652634)
('ENGLISH', 0.07231893390417099)
('ORIGINAL', 0.05214103311300278)
----
INTERNATIONAL
('ال_قانون', 0.06248722970485687)
('ال_دولي', 0.05865810811519623)
('لجنة', 0.04715884476900101)
----
LAW
('ال_قانون', 0.059738751500844955)
('ال_دولي', 0.056236479431

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [ ]:
test_en=en_model["general"]
keys=list(test_en.keys())
print(len(keys),keys[:10])
#test_ar=ar_model["العامة"]

In [ ]:
en_model["the"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 1.6295501 , -0.3641678 , -0.8005455 , -0.5281693 ,  1.0934305 ,
        0.8150351 ,  1.8593708 , -0.7521438 ,  0.4360393 , -0.5191244 ,
       -0.49931085,  1.0282642 ,  1.3791842 , -0.8435117 ,  0.40540653,
       -1.3977058 ,  0.72926307, -0.88094467, -0.7700975 , -0.29425058,
       -1.2420062 ,  1.053161  ,  0.25826624,  1.2520448 ,  1.8816774 ,
        0.00995926, -0.9422078 , -0.30660504, -1.1771382 ,  0.21316102,
       -1.1989622 ,  1.0406231 ,  1.3392975 , -0.5352807 ,  1.3703241 ,
       -0.0862059 , -0.8194424 , -2.6602635 ,  0.07741913,  1.3939295 ,
        1.1210356 , -0.7005276 , -2.4378896 ,  1.5436195 ,  1.4134588 ,
       -1.7137322 , -0.4230278 ,  1.8795353 , -1.2041483 ,  0.8313264 ,
        0.58736694,  0.07563703, -1.5684607 ,  2.3167934 ,  0.34603527,
        0.10522225,  0.5311361 , -0.02403292, -0.11283378,  2.7475295 ,
       -0.05356972, -0.05729892, -1.7050958 ,  1.0823265 ,  0.12822641,
       -1.9642165 , -0.4824889 , -1.7970736 ,  0.19963118, -0.90

In [ ]:
from itertools import groupby
def hash_tok(token0,vec_size=200):
  hash_val=abs(hash(token0))
  remainder=hash_val%vec_size
  return remainder
cur_vec_size=1000
lr=0.00001
max_val=20000
src_vec_dict={}
trg_vec_dict={}
src_counter,trg_counter={},{}

counter=0
for src0,trg0 in zip(src_sents,trg_sents):
  if counter>max_val: break
  if counter%10==0: print(counter)
  counter+=1  
  src_lower=[v.lower() for v in src0]
  trg_lower=[v.lower() for v in trg0]
  src_hashes=[hash_tok(v,cur_vec_size) for v in src_lower]
  trg_hashes=[hash_tok(v,cur_vec_size) for v in trg_lower]
  for s0 in list(set(src_lower)):
    tmp_vec=src_vec_dict.get(s0,[0]*cur_vec_size)
    new_vec=[]
    for i0,cell0 in enumerate(tmp_vec):
      cur_offset=lr#/len(trg_lower)
      if i0 in trg_hashes: cell0+=cur_offset
      else: cell0-=cur_offset
      new_vec.append(cell0)
    src_vec_dict[s0]=new_vec
  for t0 in list(set(trg_lower)):
    tmp_vec=trg_vec_dict.get(t0,[0]*cur_vec_size)
    new_vec=[]
    for i0,cell0 in enumerate(tmp_vec):
      cur_offset=lr#/len(src_lower)
      if i0 in src_hashes: cell0+=cur_offset
      else: cell0-=cur_offset
      new_vec.append(cell0)
    trg_vec_dict[t0]=new_vec

print("finished training")
counter=0
for src0,trg0 in zip(src_sents,trg_sents):
  if counter>200: break
  counter+=1

  src_lower=[v.lower() for v in src0]
  trg_lower=[v.lower() for v in trg0]
  src_hashes=[(v,hash_tok(v,cur_vec_size)) for v in src_lower]
  trg_hashes=[(v,hash_tok(v,cur_vec_size)) for v in trg_lower]
  for s0,s_hash0 in list(set(src_hashes)):
    tmp_vec=src_vec_dict.get(s0,[0]*cur_vec_size)
    print(s0)
    # print(tmp_vec)
    candidates=[]
    for t_tok0,t_hash0 in list(set(trg_hashes)):
      
      cur_val=tmp_vec[t_hash0]
      other_way_vec=trg_vec_dict[t_tok0]
      other_way_val=other_way_vec[s_hash0]
      combined=cur_val+other_way_val
      #candidates.append((t_tok0,cur_val,other_way_val,combined))
      candidates.append((t_tok0,round(combined,5)))
      #print(t_tok0,t_hash0, tmp_vec[t_hash0])
    candidates.sort(key=lambda x:x[-1],reverse=True)
    for can0 in candidates[:5]:
      print(can0)
    print("---")
  print("====")


  # src_hashes,trg_hashes=[],[]
  # for t0 in trg0:
  #   cur_hash=hash_tok(t0.lower(),cur_vec_size)
  #   trg_hashes.append(cur_hash)
  # for s0 in src0:
  #   cur_hash=hash_tok(s0.lower(),cur_vec_size)
  #   src_hashes.append(cur_hash)    
  # for s0 in src0:
  #   s_tok0=s0.lower()
  #   src_counter[s_tok0]=src_counter.get(s_tok0,0)+1
  #   tmp_vec=src_vec_dict.get(s_tok0,[0]*cur_vec_size)
  #   for th0 in trg_hashes: tmp_vec[th0]+=1/len(trg_hashes)
  #   src_vec_dict[s_tok0]=tmp_vec
  # for t0 in trg0:
  #   t_tok0=t0.lower()
  #   trg_counter[t_tok0]=trg_counter.get(t_tok0,0)+1
  #   tmp_vec=trg_vec_dict.get(t_tok0,[0]*cur_vec_size)
  #   for sh0 in src_hashes: tmp_vec[sh0]+=1/len(src_hashes) #0.0001
  #   trg_vec_dict[t_tok0]=tmp_vec

  #print("------")

# for a,b in src_vec_dict.items():
#   print(a,b)

# counter=0
# for src0,trg0 in zip(src_sents,trg_sents):
#   if counter>50: break
#   counter+=1
#   print(src0)
#   print(trg0)
#   src_hashes,trg_hashes=[],[]
#   tmp_trg_hash_dict={}
#   for t0 in trg0:
#     t_tok0=t0.lower()
#     cur_hash=hash_tok(t_tok0,cur_vec_size)
#     trg_hashes.append((t_tok0,cur_hash))
#     tmp_trg_hash_dict[t_tok0]=trg_vec_dict.get(t_tok0,[0]*cur_vec_size)

#   for s0 in src0:
#     s_tok0=s0.lower()
#     cur_hash=hash_tok(s_tok0,cur_vec_size)
#     src_hashes.append((s_tok0,cur_hash)) 
  
#   for s0 in src0:
#     s_tok0=s0.lower()
#     cur_hash=hash_tok(s_tok0,cur_vec_size)
#     cur_src_vec=src_vec_dict.get(s_tok0,[0]*cur_vec_size)
#     cur_src_vec_sum=sum(cur_src_vec)
#     cur_src_tok_count=src_counter.get(s_tok0,0)
#     print(s0)
#     candidates=[]
#     for t0_,h_hash0_ in trg_hashes:
#       t_val=cur_src_vec[h_hash0_] #round(cur_src_vec[h_hash0_],4)
#       cur_trg_tok_count=trg_counter.get(t0_,0)
#       t_val_norm=round(t_val/cur_src_vec_sum,4)
#       #t_val_norm=round(t_val/cur_trg_tok_count)

#       other_way_vec=trg_vec_dict[t0_]
#       other_way_vec_sum=sum(other_way_vec)
#       other_way_val=other_way_vec[cur_hash]#round(,4)
#       other_way_val_norm=round(other_way_val/other_way_vec_sum,4)
#       #other_way_val_norm=round(other_way_val/cur_src_tok_count)
      
#       t_val_norm=t_val
#       other_way_val_norm=other_way_val
#       #combined=round(t_val+other_way,4)
#       combined=round(t_val_norm+other_way_val_norm,6)
#       #print(t0_,combined)
#       candidates.append((t0_,combined))
#       # print(t_val,other_way)
#       # print("----")
#       #print(t0_,h_hash0_,t_val,other_way)
#     # for th0 in trg_hashes: tmp_vec[th0]+=0.0001
#     # src_vec_dict[s_tok0]=tmp_vec
#     candidates.sort(key=lambda x:-x[1])
#     for can0 in candidates[:5]:
#       print(can0)
#     print("========")
     


# collision_list=[]
# for src_toks in src_sents:
#   for a in src_toks:
#     cur_pair=(a.lower(), hash_tok(a.lower()))
#     if cur_pair in collision_list: continue
#     collision_list.append(cur_pair)
#   #print("--------")
# collision_list.sort(key=lambda x:x[1])
# for cl in collision_list:
#   print(cl)

Streaming output truncated to the last 5000 lines.
a
('.', -0.01105)
('في', -0.01401)
('بـ', -0.01879)
('فـ', -0.03486)
('حكما', -0.03676)
---
will
('سـ', 0.00172)
('ال_اختصاص', -0.00721)
('حكما', -0.00732)
('أحكاما', -0.00738)
('يتطلب', -0.00755)
---
international
('ال_دولية', 0.00138)
('حكما', -0.01019)
('ال_اختصاص', -0.01052)
('أحكاما', -0.01053)
('ال_جرائم', -0.01067)
---
which
('حكما', -0.01033)
('أحكاما', -0.01113)
('ال_اختصاص', -0.0113)
('يتطلب', -0.01192)
('ال_جرائم', -0.01197)
---
or
('أو', 0.01148)
('حكما', -0.00481)
('أحكاما', -0.00627)
('ال_اختصاص', -0.00654)
('إسناد', -0.00665)
---
jurisdiction
('ال_اختصاص', -0.00026)
('ال_جرائم', -0.00099)
('حكما', -0.00117)
('إسناد', -0.00127)
('أحكاما', -0.00141)
---
statute
('ال_أساسي', 0.00128)
('ال_نظام', -0.00069)
('حكما', -0.00088)
('ال_اختصاص', -0.00099)
('إسناد', -0.00108)
---
over
('أحكاما', -0.00154)
('حكما', -0.00156)
('إسناد', -0.00174)
('ال_اختصاص', -0.00181)
('يتطلب', -0.00203)
---
provisions
('أحكاما', -0.00101)
('حكما', -

In [ ]:
counter=0
for src0,trg0 in zip(src_sents,trg_sents):
  if counter>500: break
  counter+=1

  src_lower=[v.lower() for v in src0]
  trg_lower=[v.lower() for v in trg0]
  src_hashes=[(v,hash_tok(v,cur_vec_size)) for v in src_lower]
  trg_hashes=[(v,hash_tok(v,cur_vec_size)) for v in trg_lower]
  for s0,s_hash0 in list(set(src_hashes)):
    tmp_vec=src_vec_dict.get(s0,[0]*cur_vec_size)
    print(s0)
    # print(tmp_vec)
    candidates=[]
    for t_tok0,t_hash0 in list(set(trg_hashes)):
      cur_val=tmp_vec[t_hash0]
      other_way_vec=trg_vec_dict[t_tok0]
      other_way_val=other_way_vec[s_hash0]
      combined=cur_val+other_way_val
      if combined<0: continue
      #candidates.append((t_tok0,cur_val,other_way_val,combined))
      candidates.append((t_tok0,round(combined,5)))
      #print(t_tok0,t_hash0, tmp_vec[t_hash0])
    candidates.sort(key=lambda x:x[-1],reverse=True)
    for can0 in candidates[:5]:
      print(can0)
    print("---")
  print("====")

Streaming output truncated to the last 5000 lines.
go
---
11
---
court
('محكمة', 0.00023)
---
====
jurisidiction
('ال_ضيق', 0.0)
---
,
('.', 0.09434)
('لـ', 0.04235)
('من', 0.01974)
('أن', 0.00747)
---
all
('جميع', 0.00691)
('يوسع', 0.00078)
---
however
('يوسع', 0.00162)
('بيد', 0.00056)
---
to
('.', 0.04704)
('لـ', 0.02905)
('أن', 0.01619)
('من', 0.01318)
('يوسع', 0.00191)
---
"
('"', 0.00924)
('يوسع', 0.00032)
---
crimes
('ال_جرائم', 0.00059)
---
that
('أن', 0.02863)
('يوسع', 0.01893)
---
expands
('ال_ضيق', 0.0)
---
limited
('يوسع', 1e-05)
---
of
('.', 0.12139)
('لـ', 0.06716)
('من', 0.02661)
('أن', 0.00412)
---
an
---
.
('.', 0.18957)
('لـ', 0.06316)
('من', 0.01605)
---
report
---
character
---
the
('.', 0.15209)
('لـ', 0.0811)
('من', 0.03579)
('أن', 0.01704)
---
international
('ال_دولي', 0.00171)
---
====
would
---
such
---
before
---
how
---
trafficking
('ال_اتجار', 0.00042)
---
(
(')', 0.07117)
('(', 0.07093)
('تلك', 0.03194)
---
same
---
the
('.', 0.15209)
('وـ', 0.13902)
('لـ',

In [ ]:
span1,span2=(6, 6),(5, 39)
span1,span2=(6, 6),(7, 2)

def check_span_overlap(span1,span2):
  span1_x1,span1_x2=span1
  span2_x1,span2_x2=span2
  d1=abs(span1_x2-span1_x1)
  d2=abs(span2_x2-span2_x1)
  max_x=max(span1_x1,span1_x2,span2_x1,span2_x2)
  min_x=min(span1_x1,span1_x2,span2_x1,span2_x2)
  d_total=max_x-min_x
  overlapping=False
  if d_total<=d1+d2: overlapping=True
  return overlapping
cur_span1,cur_span2=(3, 5),(8, 12)
cur_span1,cur_span2=(3, 9),(8, 12)
cur_overlapping=check_span_overlap(cur_span1,cur_span2)
# print("d1",d1)
# print("d2",d2)
# print("d_total",d_total)
print("overlapping",cur_overlapping)

overlapping True


#Testing filtering

In [13]:
params={}
params["lang"]="en"
params["ignore_punc"]=False
params["stop_words"]=[]
params["ignore_ar_pre_suf"]=True
params["remove_al"]=False
params["index_words"]=True
params["lower"]=False
params["stemming"]=False

def filter_tokens(token_list0, params0={}):
#def filter_tokens(token_list0, lang="en",stop_words=[], ignore_punc=False,ignore_ar_pre_suf=False,remove_al=True,index_words=True,lower=True,stemming=False):
  tokens_original=list(token_list0)
  tokens_copy_enum=[(i,v) for i,v in enumerate(token_list0)]   
  ignore_punc=params0.get("ignore_punc",False)
  ignore_ar_pre_suf=params0.get("ignore_ar_pre_suf",False)
  remove_al=params0.get("remove_al",True)
  lower=params0.get("lower",True)
  stemming=params0.get("stemming",False)
  lang=params0.get("lang","en")
  stop_words=params0.get("stop_words",[])
  #if lang=="ar": tokens=tok_ar(tokens,count_dict) #tok_ar(tokens)
  if ignore_punc: tokens_copy_enum=[(i,v) for i,v in tokens_copy_enum if not is_punct(v)]
  if ignore_ar_pre_suf: tokens_copy_enum=[(i,v) for i,v in tokens_copy_enum if not v.startswith("ـ") and not v.endswith("ـ")]
  if stop_words: tokens_copy_enum=[(i,v) for i,v in tokens_copy_enum if not v.lower() in stop_words]
  if remove_al: tokens_copy_enum=[(i,v.replace("ال_","")) for i,v in tokens_copy_enum if not v.lower() in stop_words]
  tokens_out=[v[1] for v in tokens_copy_enum]
  if lower: tokens_out=[v.lower() for v in tokens_out]
  cur_mapping=[v[0] for v in tokens_copy_enum]
  return tokens_out,cur_mapping, tokens_original

#cur_ar_tok_fn(text)  
ar_corpus_fname="UNv1.0.ar-en-clean2.ar"
en_corpus_fname="UNv1.0.ar-en.en"
ar_fopen=open(ar_corpus_fname)
en_fopen=open(en_corpus_fname)
src_sents,trg_sents=[],[]
for pair_i,pair in enumerate(zip(en_fopen,ar_fopen)):
  src_line,trg_line=pair
  src_line,trg_line=src_line.strip("\n\r\t "),trg_line.strip("\n\r\t ")
  src_sents.append(src_line)
  trg_sents.append(trg_line)
  if pair_i>200: break

ar_fopen.close()
en_fopen.close()
sent_i=50
cur_sent=trg_sents[sent_i]
cur_sent=src_sents[sent_i]
cur_tokens=cur_ar_tok_fn(cur_sent)
filtered0,mapping0,original0=filter_tokens(cur_tokens, params)
print(cur_sent)
print(cur_tokens)
print(filtered0)


8. In paragraphs 442 and 443 of the report, the Working Group sets out one possible arrangement for nominating judges to serve on a court and constituting it when required.
['8', '.', 'In', 'paragraphs', '442', 'and', '443', 'of', 'the', 'report', ',', 'the', 'Working', 'Group', 'sets', 'out', 'one', 'possible', 'arrangement', 'for', 'nominating', 'judges', 'to', 'serve', 'on', 'a', 'court', 'and', 'constituting', 'it', 'when', 'required', '.']
['8', '.', 'In', 'paragraphs', '442', 'and', '443', 'of', 'the', 'report', ',', 'the', 'Working', 'Group', 'sets', 'out', 'one', 'possible', 'arrangement', 'for', 'nominating', 'judges', 'to', 'serve', 'on', 'a', 'court', 'and', 'constituting', 'it', 'when', 'required', '.']
